# Νευρωνικά Δίκτυα και Ευφυή Υπολογιστικά Συστήματα, ΣΗΜΜΥ ΕΜΠ, 2022-2023
Ομάδα 42
- Βαγγελάτος Οδυσσεύς (03118177)
- Γαλανόπουλος Νικόλαος (03118024)
- Νικολόπουλος Χρήστος (03118188)


# [🔗](https://www.kaggle.com/datasets/whenamancodes/speed-dating) Μέρος 2. Kaggle Dataset - Speed Dating

## Εισαγωγή και επισκόπηση

Το [dataset](http://www.stat.columbia.edu/~gelman/arm/examples/speed.dating/) που εξετάζουμε συλλέχθη από τους Ray Fisman & Sheena Iyengar και χρησιμοποιήθηκε για την εργασία τους με τίτλο ["Gender Differences in Mate Selection: Evidence from a Speed Dating Experiment"](https://academiccommons.columbia.edu/doi/10.7916/D8FB585Z).
Περιέχει δεδομένα που συλλέχθησαν από speed dates μεταξύ των χρόνων 2002-2004. Το [speed dating](https://en.wikipedia.org/wiki/Speed_dating) είναι μία διαδικασία “ταιριάσματος” ανθρώπων. Στόχος της είναι να ωθήσει άτομα που δεν βρίσκονται σε κάποιου είδους σχέση, να γνωρίσουν μεγάλο πλήθος από πιθανούς παρτενέρ, σε σύντομο χρονικό διάστημα.  

Στα speed dates που προαναφέραμε, οι συμμετέχοντες αρχικά γνώριζαν κάθε άλλο συμμετέχοντα του αντίθετου φύλου σε ένα “πρώτο ραντεβού”. Στη συνέχεια συμπλήρωναν ένα ερωτηματολόγιο που δήλωναν αν θα ήθελαν να ξανασυναντήσουν τον υποψήφιο παρτενέρ τους και τον αξιολογούσαν σε 6 βασικά χαρακτηριστικά: “ελκυστικότητα”, ειλικρίνια, ευφυία, πόσο ευχάριστος είναι, φιλοδοξία και τέλος κατά πόσο συμφωνούν τα ενδιαφέροντά τους. Πέρα από αυτά τα χαρακτηριστικά, περιέχονται και άλλα δεδομένα, δημογραφικού χαρακτήρα, συνήθειες στον τρόπο ζωής και συγκεκριμένα στα ραντεβού, τον τρόπο αντίληψης αξιολόγησης των βασικών χαρακτηριστικών από το φύλο τους, το αντίθετο φύλο, πως αξιολογούν τον εαυτό τους και πως πιστεύουν ό,τι αξιολογούνται οι ίδιοι κοκ. 
Οι συμμετέχοντες επανερωτούνται πολλές από αυτές τις “αξιολογήσεις” και μετά το speed dating γεγονός. Έτσι καταγράφονται ενδεχόμενες αλλαγές στις απόψεις τους.


Στόχος της παρούσας εργασίας είναι ο εξής. Χρησιμοποιώντας μόνο τις πληροφορίες που διαθέτουν και τις πεποιθησεις που έχουν οι συμμετέχοντες πριν από ένα ραντεβού και το πολύ την αξιολόγηση του ραντεβού από τους δύο συμμετέχοντές του, να “μαντέψουμε” αν ένα ζευγάρι ταιριάζει ή όχι. 

**Μετρικές**: Ως μετρικές αξιολόγησης χρησιμοποιούμε το accuracy, ώστε να αξιολογήσουμε γενικώς την ορθότητα του κάθε ταξινομητή, ανεξαρτήτως κλάσης, καθώς δεν ασχολούμαστε με κάποια κρίσιμη εφαρμογή που θα έπρεπε να αξιολογήσουμε κάποιο ρίσκο πέρα από το error rate. Ωστόσο, όπως θα διαπιστώσουμε στη συνέχεια, έχουμε ένα αρκετά imbalanced binary classification πρόβλημα. Αποφασίζουμε να χρησιμοποιήσουμε και το f1-score με `pos_label` το 1, δηλαδή το να έχουν κάνει match, δύο παρτενέρ.

**Train-test split και σχήμα CV**: Χρησιμοποιούμε συνηθισμένα σχήματα split & cv, δηλαδή απλό split αναλογίας 70%-30% και 10-fold cross validation.

Για "μερική" περιγραφή των δεδομένων του dataset, παραπέμπουμε [εδώ](http://www.stat.columbia.edu/~gelman/arm/examples/speed.dating/Speed%20Dating%20Data%20Key.doc). 

**Σημείωση:** Η περιγραφή αυτή δεν είναι αναλυτική, ενώ το dataset περιέχει και δεδομένα που δεν συμπεριλαμβάνονται στη περιγραφή αυτή, βλ. `them_cal`, `you_call`.

## Υλοποίηση

#### Environment Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA

import time

Αφού εισάγουμε το dataset με χρήση του kaggle, διαβάζουμε το δοθέν αρχείο csv με χρήση του `read_csv` της Pandas στο default encoding του, δηλαδή το "ISO-8859-1". Σε κάποια χαρακτηριστικά, όπως το `income`, τα οποία δίνονται ως strings, χρειάστηκε να τα μετατρέψουμε σε float. Προτού τα μετατρέψουμε, αφαιρούμε ενδεχόμενα "," που μπορεί να περιέχουν. πχ το "10,000.3" το μετατρέπουμε σε 10000.3 .

In [ ]:
DATASET = "../input/speed-dating/Speed Dating Data.csv"
df = pd.read_csv(DATASET, encoding = "ISO-8859-1")
X, y = df.drop(columns = 'match'), df['match']

Υπάρχουν επικεφαλίδες, αλλά όχι αρίθμηση γραμμών.


In [ ]:
print(df.head())

Οι κλάσεις που χρησιμοποιούμε για ταξινόμηση, χρησιμοποιούν το χαρακτηριστικό της κολόνας `match`. Οι κλάσεις αντιστοιχούν στο αν μεταξύ δύο συμμετεχόντων που έχουν συναντηθεί, έχουν επιλέξει και οι δύο να βγουν και άλλο ραντεβού ή όχι. 

Θα μπορούσαμε να είχαμε χρησιμοποιήσει το χαρακτηριστικό `dec` το οποίο αντιστοιχεί μόνο στο αν ο ένας από τους δύο επιλέγει να βγει και άλλο ραντεβού με τον άλλο. Έτσι θα ήταν εύκολο να ανακτήσουμε την πληροφορία του `match`, βλέποντας τη τιμή του `dec` και για τους δύο συμμετέχοντες (για τον δεύτερο το `dec_o`). Αν και οι δύο είχαν `1` στο `dec`, τότε υπάρχει `match`.
Ωστόσο η χρήση του χαρακτηριστικού `match` είναι πιο άμεση και χρειάζεται λιγότερη διαχείριση.

In [ ]:
print(f"Σύνολο τιμών των labels: {df['match'].unique()}")

In [ ]:
print(f"Πλήθος δειγμάτων: {X.shape[0]}")
print(f"Πλήθος χαρακτηριστικών: {X.shape[1]}")
print(f"Είδος χαρακτηριστικών: {X.dtypes.unique()}")
print(f"Κατηγορικά χαρακτηριστικά τύπου string: {X.select_dtypes(include = 'O').columns.tolist()}")

In [ ]:
print(f"Απουσιάζουσες τιμές? {df.isnull().any(axis = None)}")
print(f"Δείγματα που περιέχουν απουσιάζουσες τιμές: {df.isnull().any(axis = 1).sum()}")

Πρακτικά κάθε δείγμα που έχουμε έχει missing value.

In [ ]:
print(f"Πλήθος Κλάσεων: 2")
freqs = (100 * y.value_counts() / y.size).map("{:.2f}%".format)
print(f"Συχνότητα Δειγμάτων Κλάσεων:\n{freqs}")

Είναι σαφές πως το dataset μας είναι αρκετά imbalanced.

## Προετοιμασία

Διαγράφουμε τα χαρακτηριστικά που αφορούν τα ids των συμμετεχόντων και άλλα στοιχεία για τη σειρά συμμετοχής τους κλπ. στο γεγονός. Θα ασχοληθούμε με τις "γενικές απόψεις" των συμμετεχόντων πριν τα speed dates καθώς και τις απόψεις τους για τον εκάστοτε partener τους σε ένα συγκεκριμένο date. (Άρα αγνοούμε τα στοιχεία που δίδονται μετά την ημέρα των speed dates, όπως και τις "γενικές απόψεις" που δίνουν εν μέσω των dates, βλ. τα χαρακτηριστικά των οποίων το όνομα τελειώνει σε `_{s,2,3}`. Τελος αγνοούμε αν πράγματι αποφάσισε ο ένας πως επιθυμεί να ταιριάξει τον άλλο - βλ. `dec` & `dec_o`, ώστε να "αφοσιωθούμε" αποκλειστικά στον στόχο που αναφέρουμε αμέσως μετά.) Στόχος μας είναι με βάση τα εναπομείναντα χαρακτηριστικά να "μαντέψουμε" πότε δύο συμμετέχοντες ταιριάζουν (βλ. `match`). 

Δεν έχει νόημα να έχουμε ως διαθέσιμη πληροφορία τα `dec` και `dec_o` ταυτοχρόνως, διότι `match` = `dec` AND `dec_o`. Δεν θέλουμε να φτιάξουμε έναν ταξινομητή που λειτουργεί απλώς ως πύλη AND για τα δύο χαρακτηριστικά. Όπως προαναφέραμε, αποφασίζουμε να μην έχουμε κανένα από τα δύο, ώστε να αφοσιωθούμε σε προβλέψεις ταιριάσματος από πιο γενικές απόψεις και χαρακτηριστικά.

Το `from` είναι τελείως ακανόνιστο και είναι δύσκολη η διαχείριση του για να το κάνουμε καλά embed.
Αντίστοιχα και για το `undergra`. Συνεπώς, αποφασίζουμε να τα κάνουμε drop.

In [ ]:
# 'from' unique values
print(df['from'].unique())

In [ ]:
# 'undergra' unique values
print(df['undergra'].unique())

Παρακάτω απαλλασόμαστε από τα χαρακτηριστικά που προσδιορίσαμε. 

In [ ]:
## Useless columns, like id, or ones with bad format
cols = ['iid', 'id', 'idg', 'condtn', 'round', 'position', 'positin1', 'order', 'partner', 'pid',
        'samerace', 'dec_o', 'field', 'tuition', 'from', 'zipcode', 'undergra', 'career', 'dec', 
        'satis_2', 'length', 'numdat_2', 'them_cal', 'you_call']
df = df.drop(columns = cols)

## drop attr/inc/intel/fun/amb/shar{num}_s/2/3 (s=Half way through meeting, 2=day after participating, 3=weeks after)
df = df.drop(columns = df.filter(regex = "._(s|2|3)$").columns)

print("Drops completed.")

Στη συνέχεια αφαιρούμε τα waves 6-9! Η αξιολόγηση που έκαναν οι συμμετεχοντες σε χαρακτηριστικά ενδιαφέροντος (πχ. intelligence, attractiveness κλπ.) διαφέρει στα waves 6-9 σε σχέση με τα υπόλοιπα. (Στα waves 6-9 οι συμμετέχοντες βαθμολογούν όλα τα χαρακτηριστικά στο 0-10, στα υπόλοιπα waves έχουν 100 πόντους και πρέπει να τους "μοιράσουν" στα χαρακτηριστικά που τους δίδονται) Συνεπώς, για απλότητα (διότι θα είναι δύσκολο να συνδυάσουμε σε εννιαία αξιολόγηση, τις βαθμολογίες των συμμετεχόντων) κάνουμε drop τα waves 6-9 και κρατάμε τα waves 1-5 και 10-21 που σαφώς είναι πολύ περισσότερα και χρησιμοποιούν εννιαίο τρόπο βαθμολογίας. 

In [ ]:
## Ignore waves 6-9, they have different grade system
print("Original sample size: ", len(df))
df = df[(6 > df.wave) | (df.wave > 9)].drop(columns = 'wave')
print("After ignoring 6-9 waves:", len(df)) 

Κάνουμε τη μετατροπή που σχολιάσαμε στα πρώτα cells της ενότητας `Υλοποίηση`.

In [ ]:
df['income'] = df['income'].str.replace(",", "").astype(float)
df['mn_sat'] = df['mn_sat'].str.replace(",", "").astype(float)

print("Replace completed.")

Μένει να ασχοληθούμε με τη διαχείριση των κατηγορικών χαρακτηριστικών. Αρχικά τα `go_out` & `date` είναι διατεταγμένα μη κατηγορικά. 
Ελέγχοντας το dataset διαπιστώσαμε πως <180 έχουν missing values στα χαρακτηριστικά αυτά, οπότε ως ευκολότερη διαχείριση επιλέγουμε να κάνουμε drop αυτά τα δείγματα.

In [ ]:
df = df.dropna(subset = ['go_out', 'date'])

print("Drop completed.")

Για τα κατηγορικά χαρακτηριστικά που είναι μη διατεταγμένα άλλα δεν έχουν τύπο string, μετατρέπουμε τις τιμές τους σε strings.

In [ ]:
meaningless_mean_attrs = ['field_cd', 'race', 'goal', 'career_c']
df[meaningless_mean_attrs] = df[meaningless_mean_attrs].astype(str)

print("Conversion completed.")

Πλέον όλα τα μη διατεταγμένα κατηγορικά χαρακτηριστικά έχουν τύπο string, οπότε μπορούμε να κάνουμε το embed που θέλουμε με τη `get_dummies`.

In [ ]:
df = pd.get_dummies(df)

Για να κάνουμε impute των υπολοίπων missing values, κάνουμε τη παρακάτω μετατροπή:

In [ ]:
df.replace('NaN', np.NaN, inplace = True)

Κάνουμε split τα δεδομένα σε train & test.


In [ ]:
# Splitting the dataset in train and test data
train_old, test_old = train_test_split(df, test_size = .3, random_state = 42) # Remove random state.
print(f"Test split: {100 * test_old.shape[0] / df.shape[0]:.3f}%")

Παρατηρούμε πόσα missing values έχει κάθε χαρακτηριστικό. Βλέπουμε πως τα `income`, `mn_sat`, `expnum`, έχουν πάρα πολλά missing values, οπότε δεν θεωρούμε ασφαλές να τα κάνουμε impute με τη μέση τιμή που ενδεχομένως να μην είναι αρκετά αντιπροσωπευτική, οπότε απαλλασσόμαστε από αυτά τα χαρακτηριστικά.

In [ ]:
# Check if we can drop any values based on how many NaNs we see on their respective columns.
print(train_old.isnull().sum().sort_values())

train_old.drop(columns = ['expnum', 'mn_sat', 'income'], inplace=True)
test_old.drop(columns = ['expnum', 'mn_sat', 'income'], inplace=True)
# INIT COUNT = 4771
# drop expnum => dropna() gives 884
# drop expnum & mn_sat => dropna() gives 1040
# drop all above 1000 NaNs => dropna() gives 1413

Κάνουμε impute των NaN χαρακτηριστικών με βάση τις μέσες τιμές των χαρακτηριστικών του train set, ώστε να μην έχουμε καθόλου διάχυση πληροφορίας.

In [ ]:
# Fill NaNs with mean of attribute.
imp = SimpleImputer(missing_values = np.NaN, strategy = 'mean') 

# train
train = pd.DataFrame(imp.fit_transform(train_old))
train.columns = train_old.columns # Column names
train.index = train_old.index
# (same for) test
test = pd.DataFrame(imp.transform(test_old))
test.columns = test_old.columns
test.index = test_old.index

Αφού κάνουμε drop τα στοιχεία που αναφέρουμε παραπάνω, εξετάζουμε ποια από τα εναπομείναντα στοιχεία δίνουν μεγάλη συσχέτιση με το χαρακτηριστικό `match`, **στο train set**. (Η συσχέτιση δεν μπορεί να υπολογιστεί στο test set διότι θα είναι σαν να αποκτάμε πληροφορία από το test set) Αυτά είναι που θα κρατήσουμε για την εκπαίδευση των ταξινομητών μας.

In [ ]:
corr = (train.corr().dropna())['match'].abs().sort_values(ascending = False)
print(corr.unique())

THRESHOLD = .05
ind = corr.loc[corr > THRESHOLD].index.tolist()

X_train, y_train = train[ind].drop(columns = 'match'), train['match']
X_test, y_test = test[ind].drop(columns = 'match'), test['match']
print("Εναπομείναντα χαρακτηριστικά:\n", X_train.columns.values)

**Σημείωση:** Το threshold που ορίζουμε παραπάνω στο correlation θεωρείται υπερπαράμετρος στο πρόβλημα που εξετάζουμε. Ωστόσο, λόγω έλλειψης χρόνου, κατασταλλάσουμε σε ένα threshold που αφήνει ικανοποιητικό πλήθος χαρακτηριστικών που έχουν υψηλή συσχέτιση με το "μέγεθος" που θέλουμε να προβλέψουμε. (το `match` εν προκειμένω) Λόγω του μεγάλου imbalance του dataset, η απόδοση που θα δούμε παρακάτω *ίσως* να μην εξαρτάται από το ότι επιλέξαμε "λίγα" (ή ίσως και πολλά) χαρακτηριστικά σε αυτό το βήμα.

In [ ]:
# Τα παρακάτω αφορούν το "πλήρες dataset" (train & test) μετά από τη παραπάνω επεξεργασία. 
X = pd.concat([X_train, X_test], axis = 0)
y = pd.concat([y_train, y_test], axis = 0)
print(f"Dataset shape: {X.shape},",
      f"Labels shape: {y.shape}")
print(f"Unique datatypes: {X.dtypes.unique()}")
print(f"Unknown values? {X.isnull().values.any()}")
print(f"Classes: {y.nunique()}")
freqs = (100 * y.value_counts() / y.size).map("{:.2f}%".format)
print(f"Class Frequencies:\n{freqs}")

Τυπώνουμε ενδεικτικά τις διακυμάνσεις των χαρακτηριστικών στο test set, ώστε να τις χρησιμοποιήσουμε ως γνώμονα στο `VarianceThreshold` κατά τη βελτιστοποίηση των ταξινομητών.

In [ ]:
print(X_test.var())

## Ταξινόμηση

#### Out-of-the-box

In [ ]:
clfs = [DummyClassifier(), MLPClassifier(), SVC()]

In [ ]:
f1s, accs = [], []  # F1-score(s), accuracy(s)
acc_preds = []
for clf in clfs:
    acc_preds.append(clf.fit(X_train, y_train).predict(X_test))
    accs.append(100 * accuracy_score(y_test, acc_preds[-1]))
    f1s.append(100 * f1_score(y_test, acc_preds[-1])) # pos_label = 1: Match! 
    print(str(clf)[:-2], f"Accuracy: {accs[-1]:.3f}%,",
         f"F1-score: {f1s[-1]:.3f}%")

| **Metric**  | **Dummy** | **MLP** | **SVM** |
|:-----------:|:---------:|:-------:|:-------:|
| Accuracy (%)| 83.994    | 85.976  | 84.743  |
| F1-Score (%)| 0.000     | 46.503  | 19.444  |

**Σημείωση**: Από εκτέλεση σε εκτέλεση, η επίδοση του MLP μπορεί να ποικίλει, ιδιαίτερα στο `f1` score, λόγω της τυχαίοτητας στην εκπαίδευση του MLP με stochastic gradient descent και "τυχαία" αρχικοποίηση, ωστόσο σε γενικές γραμμές αποδίδει καλύτερα και στις δύο μετρικές και από τους άλλους 2 ταξινομητές.

Λόγω του μεγάλου imbalance βλέπουμε πως όλοι οι ταξινομητές έχουν πολύ μεγάλο accuracy. Ωστόσο, αξιολογώντας το `f1` score ως προς την υπολοιπώμενη κλάση (`match` = 1) βλέπουμε πως ο dummy έχει μηδενικό `f1`, το SVM αρκετά χαμηλό σκορ κοντά στο 20% ενώ το ootb MLP σχετικά ικανοποιητικό `f1` κοντά στο 46%.

In [ ]:
names = ['dummy', 'MLP', 'SVM']
# Metrics of one execution (check the table above). MLP Metrics are stochastic.
accs = [83.994, 85.976, 84.473] 
f1 = [0, 46.503, 19.444]
fig, ax = plt.subplots(nrows = 2)
ax[0].set_ylabel("Accuracy (%)")
ax[1].set_ylabel("F1-Score (%)")
ax[0].bar(names, accs)
ax[1].bar(names, f1s)
plt.show()

Αναμένουμε οι βελτιώσεις κατά τη βελτιστοποίηση να μην είναι πολύ σημαντικές στο `accuracy` λόγω του μεγάλου imbalance και των ήδη μεγάλων τιμών της μετρικής αυτής, ενώ στο `f1` score, αναμένουμε να δούμε βελτιώσεις, χωρίς ωστόσο να μπορούμε να πετύχουμε `scores` της τάξης του 80% ή παραπάνω, λόγω της "προβληματικής φύσης" του dataset.

### Βελτιστοποίηση

#### MLP Optimization

Σε πρώτη φάση, αυξάνουμε τα `max_iters` του MLP από 200 σε 1000. Δεν ασχολούμαστε με τους διαθέσιμους solvers, ενώ εξετάζουμε αρκετά μικρότερο πλήθος νευρώνων στα παρακάτω νευρωνικά, κάτι που σίγουρα μειώνει τον χρόνο εκπαίδευσης αλλά ενδεχομένως και την απόδοσή τους. (Εκτός αν το μεγάλο πλήθος νευρώνων υπερέβαινε τη "χωρητικότητα" του προβλήματος.) Τέλος, δεν φιξάρουμε τη "τυχαία αρχικοποίηση" του MLP με κάποιο seed, διότι δεν είθισται, αν και αυτό θα κάνει πιο απρόβλεπτα τα αποτελέσματα παρακάτω και ενδεχομένως να οδηγήσει σε ασυνεπή συμπεράσματα.

#### Accuracy

Δοκιμάσαμε ενδεικτικά αρχιτεκτονικές χωρίς sampler και αντίστοιχα χωρίς scaler, όμως με κάποιες εκτελέσεις διαπιστώσαμε πως δεν απέδιδαν καλά. Συνεπώς, αρκούμαστε στο να παρουσιάσουμε τη "πλήρη" αρχιτεκτονική.

In [ ]:
def nn_accuracy(trial):
    threshold = trial.suggest_int("var_threshold", 0, 3)
    selector = VarianceThreshold(threshold = threshold)

    scaler = StandardScaler() 
    ros = RandomOverSampler() 

    n_components = trial.suggest_int('n_components', 1, 15)
    pca = PCA(n_components)

    n_layers = trial.suggest_int('n_layers', 1, 2)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units{i}', 10, 20))

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)

    pipe = make_pipeline(scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring='accuracy')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_accuracy, n_trials = 20)

In [ ]:
# I 2022-12-06 07:38:51,609] A new study created in memory with name: no-name-8aa613e5-afd4-4cd2-baca-38ea81b5e80b
# [I 2022-12-06 07:39:30,193] Trial 0 finished with value: 0.749255545015133 and parameters: {'var_threshold': 1, 'n_components': 7, 'n_layers': 1, 'n_units_0': 16}. Best is trial 0 with value: 0.749255545015133.
# [I 2022-12-06 07:39:42,917] Trial 1 finished with value: 0.7469232506663053 and parameters: {'var_threshold': 3, 'n_components': 2, 'n_layers': 1, 'n_units_0': 20}. Best is trial 0 with value: 0.749255545015133.
# [I 2022-12-06 07:40:53,407] Trial 2 finished with value: 0.7532922256855039 and parameters: {'var_threshold': 1, 'n_components': 8, 'n_layers': 2, 'n_units_0': 10, 'n_units_1': 15}. Best is trial 2 with value: 0.7532922256855039.
# [I 2022-12-06 07:41:08,390] Trial 3 finished with value: 0.7511668247730044 and parameters: {'var_threshold': 3, 'n_components': 3, 'n_layers': 1, 'n_units_0': 15}. Best is trial 2 with value: 0.7532922256855039.
# [I 2022-12-06 07:41:23,131] Trial 4 finished with value: 0.7481944256222615 and parameters: {'var_threshold': 2, 'n_components': 3, 'n_layers': 1, 'n_units_0': 17}. Best is trial 2 with value: 0.7532922256855039.
# [I 2022-12-06 07:42:28,288] Trial 5 finished with value: 0.7596648145638524 and parameters: {'var_threshold': 1, 'n_components': 4, 'n_layers': 2, 'n_units_0': 16, 'n_units_1': 20}. Best is trial 5 with value: 0.7596648145638524.
# [I 2022-12-06 07:42:58,808] Trial 6 finished with value: 0.7430957220942314 and parameters: {'var_threshold': 3, 'n_components': 2, 'n_layers': 2, 'n_units_0': 16, 'n_units_1': 13}. Best is trial 5 with value: 0.7596648145638524.
# [I 2022-12-06 07:43:17,607] Trial 7 finished with value: 0.7501034467181642 and parameters: {'var_threshold': 3, 'n_components': 3, 'n_layers': 1, 'n_units_0': 18}. Best is trial 5 with value: 0.7596648145638524.
# [I 2022-12-06 07:44:14,458] Trial 8 finished with value: 0.7692383791841713 and parameters: {'var_threshold': 1, 'n_components': 14, 'n_layers': 1, 'n_units_0': 17}. Best is trial 8 with value: 0.7692383791841713.
# [I 2022-12-06 07:45:10,478] Trial 9 finished with value: 0.7728558521931609 and parameters: {'var_threshold': 2, 'n_components': 12, 'n_layers': 1, 'n_units_0': 19}. Best is trial 9 with value: 0.7728558521931609.
# [I 2022-12-06 07:46:34,217] Trial 10 finished with value: 0.7734819532908704 and parameters: {'var_threshold': 0, 'n_components': 14, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 10}. Best is trial 10 with value: 0.7734819532908704.
# [I 2022-12-06 07:47:55,918] Trial 11 finished with value: 0.7762488142024665 and parameters: {'var_threshold': 0, 'n_components': 14, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 10}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:49:23,303] Trial 12 finished with value: 0.7568952432578941 and parameters: {'var_threshold': 0, 'n_components': 15, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 10}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:50:37,422] Trial 13 finished with value: 0.7700808600984776 and parameters: {'var_threshold': 0, 'n_components': 11, 'n_layers': 2, 'n_units_0': 12, 'n_units_1': 10}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:51:54,984] Trial 14 finished with value: 0.7690116095225189 and parameters: {'var_threshold': 0, 'n_components': 11, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 12}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:53:11,208] Trial 15 finished with value: 0.763283642770023 and parameters: {'var_threshold': 0, 'n_components': 13, 'n_layers': 2, 'n_units_0': 11, 'n_units_1': 10}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:54:36,826] Trial 16 finished with value: 0.7688078782129467 and parameters: {'var_threshold': 0, 'n_components': 10, 'n_layers': 2, 'n_units_0': 14, 'n_units_1': 13}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:56:11,106] Trial 17 finished with value: 0.7709237927451777 and parameters: {'var_threshold': 0, 'n_components': 15, 'n_layers': 2, 'n_units_0': 12, 'n_units_1': 18}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:57:30,196] Trial 18 finished with value: 0.7688164611284275 and parameters: {'var_threshold': 2, 'n_components': 9, 'n_layers': 2, 'n_units_0': 14, 'n_units_1': 12}. Best is trial 11 with value: 0.7762488142024665.
# [I 2022-12-06 07:58:30,464] Trial 19 finished with value: 0.7577553417355558 and parameters: {'var_threshold': 1, 'n_components': 6, 'n_layers': 2, 'n_units_0': 10, 'n_units_1': 16}. Best is trial 11 with value: 0.7762488142024665.

Παρατηρούμε πως για n_layers = 2, έχουμε καλύτερη επίδοση από ότι στις εκτελέσεις με 1 κρυφό επίπεδο.

In [ ]:
def nn_accuracy(trial):
    threshold = trial.suggest_float("var_threshold", 1.5, 3.5, step = 0.2)
    selector = VarianceThreshold(threshold = threshold)

    scaler = StandardScaler() 
    ros = RandomOverSampler() 

    n_components = trial.suggest_int('n_components', 1, 15)
    pca = PCA(n_components)

    n_layers = 2
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units{i}', 10, 20))

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)

    pipe = make_pipeline(scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring='accuracy')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_accuracy, n_trials = 10)

In [ ]:
# [I 2022-12-06 07:59:22,572] A new study created in memory with name: no-name-a1814d23-e8c9-4bbf-9e92-02ea8d5aacc0
# [I 2022-12-06 08:00:17,363] Trial 0 finished with value: 0.7603085332249175 and parameters: {'var_threshold': 2.9000000000000004, 'n_components': 10, 'n_units_0': 10, 'n_units_1': 11}. Best is trial 0 with value: 0.7603085332249175.
# [I 2022-12-06 08:01:22,820] Trial 1 finished with value: 0.7420300853774224 and parameters: {'var_threshold': 1.7, 'n_components': 7, 'n_units_0': 10, 'n_units_1': 11}. Best is trial 0 with value: 0.7603085332249175.
# [I 2022-12-06 08:02:11,638] Trial 2 finished with value: 0.7594538555359805 and parameters: {'var_threshold': 1.9, 'n_components': 4, 'n_units_0': 14, 'n_units_1': 10}. Best is trial 0 with value: 0.7603085332249175.
# [I 2022-12-06 08:03:34,173] Trial 3 finished with value: 0.7505389167457199 and parameters: {'var_threshold': 1.7, 'n_components': 8, 'n_units_0': 14, 'n_units_1': 13}. Best is trial 0 with value: 0.7603085332249175.
# [I 2022-12-06 08:05:10,971] Trial 4 finished with value: 0.7660477932872567 and parameters: {'var_threshold': 2.5, 'n_components': 8, 'n_units_0': 18, 'n_units_1': 20}. Best is trial 4 with value: 0.7660477932872567.
# [I 2022-12-06 08:06:26,244] Trial 5 finished with value: 0.7571170438632153 and parameters: {'var_threshold': 1.5, 'n_components': 6, 'n_units_0': 12, 'n_units_1': 18}. Best is trial 4 with value: 0.7660477932872567.
# [I 2022-12-06 08:07:41,115] Trial 6 finished with value: 0.7649758323169354 and parameters: {'var_threshold': 2.3, 'n_components': 10, 'n_units_0': 12, 'n_units_1': 16}. Best is trial 4 with value: 0.7660477932872567.
# [I 2022-12-06 08:09:08,928] Trial 7 finished with value: 0.7700754393097531 and parameters: {'var_threshold': 1.5, 'n_components': 15, 'n_units_0': 12, 'n_units_1': 16}. Best is trial 7 with value: 0.7700754393097531.
# [I 2022-12-06 08:10:53,214] Trial 8 finished with value: 0.7637055608257668 and parameters: {'var_threshold': 2.9000000000000004, 'n_components': 10, 'n_units_0': 17, 'n_units_1': 15}. Best is trial 7 with value: 0.7700754393097531.
# [I 2022-12-06 08:10:59,676] Trial 9 finished with value: 0.7422442065320505 and parameters: {'var_threshold': 3.1, 'n_components': 1, 'n_units_0': 17, 'n_units_1': 19}. Best is trial 7 with value: 0.7700754393097531.

Παραπάνω εξετάσαμε πιο λεπτομερώς μικρό διάστημα τιμών για το `var_threshold`. 
Με βάση τις εκτελέσεις που προκύπτουν, το περιορίζουμε στο [1.5, 2.9] και ελέγχουμε πιο συστηματικά την επίδραση των components του PCA.

In [ ]:
def nn_accuracy(trial):
    threshold = trial.suggest_float("var_threshold", 1.5, 2.9, step=0.2)
    selector = VarianceThreshold(threshold = threshold)

    scaler = StandardScaler() 
    ros = RandomOverSampler() 

    n_components = trial.suggest_int('n_components', 9, 15)
    pca = PCA(n_components)

    n_layers = 2
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'nunits{i}', 10, 20))

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)

    pipe = make_pipeline(scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring='accuracy')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_accuracy, n_trials = 10)

In [ ]:
# [I 2022-12-06 08:48:18,652] A new study created in memory with name: no-name-7762450a-e378-4d41-a36d-3ffe136423e9
# [I 2022-12-06 08:49:55,178] Trial 0 finished with value: 0.762852238334011 and parameters: {'var_threshold': 2.3, 'n_components': 9, 'n_units_0': 16, 'n_units_1': 16}. Best is trial 0 with value: 0.762852238334011.
# [I 2022-12-06 08:51:33,719] Trial 1 finished with value: 0.7728445588833175 and parameters: {'var_threshold': 2.3, 'n_components': 10, 'n_units_0': 20, 'n_units_1': 16}. Best is trial 1 with value: 0.7728445588833175.
# [I 2022-12-06 08:52:56,865] Trial 2 finished with value: 0.7677422414961377 and parameters: {'var_threshold': 2.1, 'n_components': 9, 'n_units_0': 15, 'n_units_1': 14}. Best is trial 1 with value: 0.7728445588833175.
# [I 2022-12-06 08:54:09,064] Trial 3 finished with value: 0.7617852464200208 and parameters: {'var_threshold': 1.5, 'n_components': 11, 'n_units_0': 12, 'n_units_1': 11}. Best is trial 1 with value: 0.7728445588833175.
# [I 2022-12-06 08:55:37,386] Trial 4 finished with value: 0.7686023399737996 and parameters: {'var_threshold': 2.1, 'n_components': 11, 'n_units_0': 19, 'n_units_1': 12}. Best is trial 1 with value: 0.7728445588833175.
# [I 2022-12-06 08:57:21,568] Trial 5 finished with value: 0.7628572073903419 and parameters: {'var_threshold': 1.7, 'n_components': 13, 'n_units_0': 12, 'n_units_1': 20}. Best is trial 1 with value: 0.7728445588833175.
# [I 2022-12-06 08:58:53,757] Trial 6 finished with value: 0.7753905226543796 and parameters: {'var_threshold': 2.5, 'n_components': 10, 'n_units_0': 18, 'n_units_1': 12}. Best is trial 6 with value: 0.7753905226543796.
# [I 2022-12-06 08:00:26,122] Trial 7 finished with value: 0.7694416587613497 and parameters: {'var_threshold': 1.7, 'n_components': 14, 'n_units_0': 15, 'n_units_1': 10}. Best is trial 6 with value: 0.7753905226543796.
# [I 2022-12-06 08:01:49,357] Trial 8 finished with value: 0.7741170890364548 and parameters: {'var_threshold': 2.3, 'n_components': 14, 'n_units_0': 14, 'n_units_1': 16}. Best is trial 6 with value: 0.7753905226543796.
# [I 2022-12-06 08:03:19,518] Trial 9 finished with value: 0.7855933504991643 and parameters: {'var_threshold': 2.9, 'n_components': 14, 'n_units_0': 20, 'n_units_1': 12}. Best is trial 9 with value: 0.7855933504991643.

Καταλήγουμε στην αρχιτεκτονική του τελευταίου trial, δηλαδή selector με threshold 2.9, standard scaler,  random over sampler, PCA 14 συνιστωσών και έναν νευρωνικό δύο κρυφών επιπέδων, με 20 νευρώνες στο πρώτο επίπεδο και 12 στο δεύτερο.

In [ ]:
threshold = VarianceThreshold(threshold = 2.9)
scaler = StandardScaler()
sampler = RandomOverSampler()
pca = PCA(n_components = 14)
clf = MLPClassifier(hidden_layer_sizes = tuple([20, 12]), max_iter = 1000)

pipe = make_pipeline(scaler, sampler, pca, clf)


start_time = time.time()
pipe.fit(X_train, y_train)
print(f"Time spent to fit: {time.time() - start_time:.6f} seconds")  

start_time = time.time()  
mlp_acc_preds = pipe.predict(X_test)
print(f"Time spent to test: {time.time() - start_time:.6f} seconds")

print(f"Accuracy on test set: {100 * accuracy_score(y_test, mlp_acc_preds):.3f}%")

#### F1 Score

Επιλέγουμε να χρησιμοποιήσουμε Variance Threshold με μικρές τιμές στο variance threshold, ώστε να μην σβήσουμε τα ισχυρά συσχετιζόμενα (με το match) χαρακτηριστικά.

In [ ]:
def nn_f1(trial):
    threshold = trial.suggest_int("var_threshold", 0, 3)
    selector = VarianceThreshold(threshold = threshold)
    
    scaler = StandardScaler()
    ros = RandomOverSampler()

    n_components = trial.suggest_int('n_components', 1, 15)
    pca = PCA(n_components)
    
    n_layers = trial.suggest_int('n_layers', 1, 2)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units_{i}', 10, 20))

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)
    
    pipe = make_pipeline(scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring = 'f1')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_f1, n_trials = 20)

In [ ]:
# [I 2022-11-27 08:15:35,732] A new study created in memory with name: no-name-26e1af2f-b24e-467d-9cc0-505032de3c86
# [I 2022-11-27 08:18:04,261] Trial 0 finished with value: 0.5037441693998966 and parameters: {'var_threshold': 3, 'n_components': 14, 'n_layers': 2, 'n_units_0': 14, 'n_units_1': 14}. Best is trial 0 with value: 0.5037441693998966.
# [I 2022-11-27 08:18:57,093] Trial 1 finished with value: 0.5089098054499746 and parameters: {'var_threshold': 1, 'n_components': 8, 'n_layers': 1, 'n_units_0': 16}. Best is trial 1 with value: 0.5089098054499746.
# [I 2022-11-27 08:20:13,611] Trial 2 finished with value: 0.5126184551455368 and parameters: {'var_threshold': 3, 'n_components': 14, 'n_layers': 1, 'n_units_0': 16}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:20:21,702] Trial 3 finished with value: 0.5039828268915265 and parameters: {'var_threshold': 1, 'n_components': 1, 'n_layers': 2, 'n_units_0': 15, 'n_units_1': 16}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:21:41,918] Trial 4 finished with value: 0.5085777626354422 and parameters: {'var_threshold': 3, 'n_components': 3, 'n_layers': 2, 'n_units_0': 17, 'n_units_1': 17}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:23:30,002] Trial 5 finished with value: 0.49655533017366676 and parameters: {'var_threshold': 2, 'n_components': 10, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 11}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:24:29,879] Trial 6 finished with value: 0.5051924205938314 and parameters: {'var_threshold': 1, 'n_components': 7, 'n_layers': 1, 'n_units_0': 18}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:26:14,107] Trial 7 finished with value: 0.49775769762420036 and parameters: {'var_threshold': 0, 'n_components': 5, 'n_layers': 2, 'n_units_0': 15, 'n_units_1': 16}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:28:20,127] Trial 8 finished with value: 0.4932343517992376 and parameters: {'var_threshold': 2, 'n_components': 11, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 19}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:30:43,126] Trial 9 finished with value: 0.4874855478937922 and parameters: {'var_threshold': 0, 'n_components': 9, 'n_layers': 2, 'n_units_0': 20, 'n_units_1': 16}. Best is trial 2 with value: 0.5126184551455368.
# [I 2022-11-27 08:31:30,819] Trial 10 finished with value: 0.517443197876685 and parameters: {'var_threshold': 3, 'n_components': 15, 'n_layers': 1, 'n_units_0': 11}. Best is trial 10 with value: 0.517443197876685.
# [I 2022-11-27 08:32:19,115] Trial 11 finished with value: 0.5110549276376604 and parameters: {'var_threshold': 3, 'n_components': 15, 'n_layers': 1, 'n_units_0': 10}. Best is trial 10 with value: 0.517443197876685.
# [I 2022-11-27 08:33:04,150] Trial 12 finished with value: 0.5091372745629782 and parameters: {'var_threshold': 2, 'n_components': 13, 'n_layers': 1, 'n_units_0': 10}. Best is trial 10 with value: 0.517443197876685.
# [I 2022-11-27 08:33:52,978] Trial 13 finished with value: 0.5211303411872796 and parameters: {'var_threshold': 3, 'n_components': 12, 'n_layers': 1, 'n_units_0': 12}. Best is trial 13 with value: 0.5211303411872796.
# [I 2022-11-27 08:34:47,943] Trial 14 finished with value: 0.5097359485609755 and parameters: {'var_threshold': 3, 'n_components': 12, 'n_layers': 1, 'n_units_0': 12}. Best is trial 13 with value: 0.5211303411872796.
# [I 2022-11-27 08:35:45,145] Trial 15 finished with value: 0.5213704167099354 and parameters: {'var_threshold': 2, 'n_components': 15, 'n_layers': 1, 'n_units_0': 11}. Best is trial 15 with value: 0.5213704167099354.
# [I 2022-11-27 08:36:34,055] Trial 16 finished with value: 0.5202708375121636 and parameters: {'var_threshold': 2, 'n_components': 12, 'n_layers': 1, 'n_units_0': 12}. Best is trial 15 with value: 0.5213704167099354.
# [I 2022-11-27 08:37:17,870] Trial 17 finished with value: 0.5130958391423721 and parameters: {'var_threshold': 2, 'n_components': 11, 'n_layers': 1, 'n_units_0': 11}. Best is trial 15 with value: 0.5213704167099354.
# [I 2022-11-27 08:37:56,605] Trial 18 finished with value: 0.5168077756158602 and parameters: {'var_threshold': 2, 'n_components': 6, 'n_layers': 1, 'n_units_0': 13}. Best is trial 15 with value: 0.5213704167099354.
# [I 2022-11-27 08:38:50,472] Trial 19 finished with value: 0.5092194510772284 and parameters: {'var_threshold': 1, 'n_components': 13, 'n_layers': 1, 'n_units_0': 11}. Best is trial 15 with value: 0.5213704167099354.

In [ ]:
# [I 2022-11-27 08:41:58,909] A new study created in memory with name: no-name-763c03e8-cb52-4f26-b5a9-2888ead680fc
# [I 2022-11-27 08:42:31,259] Trial 0 finished with value: 0.5159235943965385 and parameters: {'var_threshold': 0, 'n_components': 6, 'n_layers': 1, 'n_units_0': 12}. Best is trial 0 with value: 0.5159235943965385.
# [I 2022-11-27 08:43:27,131] Trial 1 finished with value: 0.5238032753910609 and parameters: {'var_threshold': 3, 'n_components': 10, 'n_layers': 1, 'n_units_0': 12}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:44:27,263] Trial 2 finished with value: 0.5221883474153971 and parameters: {'var_threshold': 1, 'n_components': 11, 'n_layers': 1, 'n_units_0': 14}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:45:05,219] Trial 3 finished with value: 0.5096555986066396 and parameters: {'var_threshold': 0, 'n_components': 4, 'n_layers': 1, 'n_units_0': 19}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:45:47,704] Trial 4 finished with value: 0.5173102693687175 and parameters: {'var_threshold': 3, 'n_components': 4, 'n_layers': 1, 'n_units_0': 20}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:46:33,164] Trial 5 finished with value: 0.5045469608681523 and parameters: {'var_threshold': 3, 'n_components': 7, 'n_layers': 1, 'n_units_0': 15}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:47:39,289] Trial 6 finished with value: 0.5091113026592331 and parameters: {'var_threshold': 1, 'n_components': 11, 'n_layers': 1, 'n_units_0': 14}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:49:02,821] Trial 7 finished with value: 0.514927482215825 and parameters: {'var_threshold': 0, 'n_components': 12, 'n_layers': 1, 'n_units_0': 20}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:49:22,052] Trial 8 finished with value: 0.5081755415396673 and parameters: {'var_threshold': 3, 'n_components': 5, 'n_layers': 1, 'n_units_0': 10}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:51:09,077] Trial 9 finished with value: 0.49235965172737284 and parameters: {'var_threshold': 3, 'n_components': 4, 'n_layers': 2, 'n_units_0': 20, 'n_units_1': 20}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:53:19,267] Trial 10 finished with value: 0.49470370184595847 and parameters: {'var_threshold': 2, 'n_components': 15, 'n_layers': 2, 'n_units_0': 17, 'n_units_1': 10}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:55:16,392] Trial 11 finished with value: 0.4940329138110068 and parameters: {'var_threshold': 1, 'n_components': 10, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 15}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:56:00,880] Trial 12 finished with value: 0.5160527452485517 and parameters: {'var_threshold': 2, 'n_components': 9, 'n_layers': 1, 'n_units_0': 11}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:57:14,615] Trial 13 finished with value: 0.517589136957476 and parameters: {'var_threshold': 1, 'n_components': 13, 'n_layers': 1, 'n_units_0': 16}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:57:23,621] Trial 14 finished with value: 0.5063694706327118 and parameters: {'var_threshold': 2, 'n_components': 1, 'n_layers': 2, 'n_units_0': 13, 'n_units_1': 10}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:58:17,228] Trial 15 finished with value: 0.5168348305747392 and parameters: {'var_threshold': 2, 'n_components': 15, 'n_layers': 1, 'n_units_0': 10}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 08:59:18,817] Trial 16 finished with value: 0.5142081377184751 and parameters: {'var_threshold': 1, 'n_components': 8, 'n_layers': 1, 'n_units_0': 17}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 09:01:35,913] Trial 17 finished with value: 0.4939663282022555 and parameters: {'var_threshold': 2, 'n_components': 13, 'n_layers': 2, 'n_units_0': 12, 'n_units_1': 20}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 09:02:37,817] Trial 18 finished with value: 0.5228054996594309 and parameters: {'var_threshold': 1, 'n_components': 10, 'n_layers': 1, 'n_units_0': 14}. Best is trial 1 with value: 0.5238032753910609.
# [I 2022-11-27 09:03:36,816] Trial 19 finished with value: 0.5163678680564767 and parameters: {'var_threshold': 0, 'n_components': 9, 'n_layers': 1, 'n_units_0': 15}. Best is trial 1 with value: 0.5238032753910609.

Με βάση τις παραπάνω ενδεικτικές εκτελέσεις, παρατηρούμε πως μάλλον αξίζει να χρησιμοποιήσουμε ένα μόνο layer νευρώνων.
Θα επιχειρήσουμε να αναζητήσουμε όριο στο Variance Threshold, στο διάστημα [2-4] με βήμα 0.2.

In [ ]:
def nn_f1(trial):
    threshold = trial.suggest_float("var_threshold", 2., 4., step = 0.2)
    selector = VarianceThreshold(threshold = threshold)
    
    scaler = StandardScaler()
    ros = RandomOverSampler()

    n_components = trial.suggest_int('n_components', 1, 15)
    pca = PCA(n_components)
    
    layers = [trial.suggest_int(f'n_units_0', 10, 20)]

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)
    
    pipe = make_pipeline(scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring = 'f1')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_f1, n_trials = 20)

In [ ]:
# [I 2022-11-27 09:08:41,826] A new study created in memory with name: no-name-680e8e8c-2ddc-4d63-aa3f-a47d72e56800
# [I 2022-11-27 09:09:20,488] Trial 0 finished with value: 0.5085492346987689 and parameters: {'var_threshold': 2.8, 'n_components': 6, 'n_units_0': 13}. Best is trial 0 with value: 0.5085492346987689.
# [I 2022-11-27 09:09:26,800] Trial 1 finished with value: 0.5043149518339622 and parameters: {'var_threshold': 2.2, 'n_components': 1, 'n_units_0': 12}. Best is trial 0 with value: 0.5085492346987689.
# [I 2022-11-27 09:10:23,875] Trial 2 finished with value: 0.5195674972919733 and parameters: {'var_threshold': 2.6, 'n_components': 9, 'n_units_0': 15}. Best is trial 2 with value: 0.5195674972919733.
# [I 2022-11-27 09:11:36,896] Trial 3 finished with value: 0.5133037878789685 and parameters: {'var_threshold': 2.0, 'n_components': 14, 'n_units_0': 16}. Best is trial 2 with value: 0.5195674972919733.
# [I 2022-11-27 09:11:53,268] Trial 4 finished with value: 0.5041225269651526 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 2, 'n_units_0': 13}. Best is trial 2 with value: 0.5195674972919733.
# [I 2022-11-27 09:12:21,707] Trial 5 finished with value: 0.5109785909963994 and parameters: {'var_threshold': 3.6, 'n_components': 4, 'n_units_0': 12}. Best is trial 2 with value: 0.5195674972919733.
# [I 2022-11-27 09:13:12,716] Trial 6 finished with value: 0.524018237641591 and parameters: {'var_threshold': 2.0, 'n_components': 5, 'n_units_0': 20}. Best is trial 6 with value: 0.524018237641591.
# [I 2022-11-27 09:14:01,353] Trial 7 finished with value: 0.517803373439612 and parameters: {'var_threshold': 3.0, 'n_components': 13, 'n_units_0': 10}. Best is trial 6 with value: 0.524018237641591.
# [I 2022-11-27 09:14:07,949] Trial 8 finished with value: 0.5061792636625647 and parameters: {'var_threshold': 2.8, 'n_components': 1, 'n_units_0': 17}. Best is trial 6 with value: 0.524018237641591.
# [I 2022-11-27 09:14:29,686] Trial 9 finished with value: 0.5098852877221808 and parameters: {'var_threshold': 2.4, 'n_components': 3, 'n_units_0': 13}. Best is trial 6 with value: 0.524018237641591.
# [I 2022-11-27 09:15:46,962] Trial 10 finished with value: 0.514685690657888 and parameters: {'var_threshold': 4.0, 'n_components': 9, 'n_units_0': 20}. Best is trial 6 with value: 0.524018237641591.
# [I 2022-11-27 09:17:03,144] Trial 11 finished with value: 0.5314930458225071 and parameters: {'var_threshold': 2.4, 'n_components': 9, 'n_units_0': 19}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:18:10,477] Trial 12 finished with value: 0.5080758178371045 and parameters: {'var_threshold': 2.0, 'n_components': 7, 'n_units_0': 20}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:19:32,135] Trial 13 finished with value: 0.5144228433107367 and parameters: {'var_threshold': 2.4, 'n_components': 11, 'n_units_0': 18}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:20:32,781] Trial 14 finished with value: 0.5120533521446162 and parameters: {'var_threshold': 2.0, 'n_components': 6, 'n_units_0': 19}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:21:45,756] Trial 15 finished with value: 0.5136389102003837 and parameters: {'var_threshold': 2.4, 'n_components': 11, 'n_units_0': 18}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:22:38,469] Trial 16 finished with value: 0.515169163425558 and parameters: {'var_threshold': 3.2, 'n_components': 5, 'n_units_0': 20}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:23:55,415] Trial 17 finished with value: 0.5201178097223541 and parameters: {'var_threshold': 2.2, 'n_components': 11, 'n_units_0': 18}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:24:41,894] Trial 18 finished with value: 0.5068429842829674 and parameters: {'var_threshold': 2.6, 'n_components': 8, 'n_units_0': 15}. Best is trial 11 with value: 0.5314930458225071.
# [I 2022-11-27 09:25:21,160] Trial 19 finished with value: 0.5196831770122403 and parameters: {'var_threshold': 2.2, 'n_components': 4, 'n_units_0': 19}. Best is trial 11 with value: 0.5314930458225071.

In [ ]:
# [I 2022-11-27 09:27:28,912] A new study created in memory with name: no-name-c000c4e5-3585-4344-8aed-18ae98231a54
# [I 2022-11-27 09:27:52,279] Trial 0 finished with value: 0.5155917144057973 and parameters: {'var_threshold': 3.8, 'n_components': 4, 'n_units_0': 10}. Best is trial 0 with value: 0.5155917144057973.
# [I 2022-11-27 09:28:31,492] Trial 1 finished with value: 0.5159425101307523 and parameters: {'var_threshold': 2.2, 'n_components': 5, 'n_units_0': 15}. Best is trial 1 with value: 0.5159425101307523.
# [I 2022-11-27 09:29:07,722] Trial 2 finished with value: 0.506407651000592 and parameters: {'var_threshold': 4.0, 'n_components': 8, 'n_units_0': 10}. Best is trial 1 with value: 0.5159425101307523.
# [I 2022-11-27 09:29:26,172] Trial 3 finished with value: 0.5119567450790334 and parameters: {'var_threshold': 3.2, 'n_components': 3, 'n_units_0': 12}. Best is trial 1 with value: 0.5159425101307523.
# [I 2022-11-27 09:30:10,897] Trial 4 finished with value: 0.5162647959075072 and parameters: {'var_threshold': 4.0, 'n_components': 5, 'n_units_0': 15}. Best is trial 4 with value: 0.5162647959075072.
# [I 2022-11-27 09:30:51,208] Trial 5 finished with value: 0.5183728839384922 and parameters: {'var_threshold': 2.4, 'n_components': 10, 'n_units_0': 10}. Best is trial 5 with value: 0.5183728839384922.
# [I 2022-11-27 09:31:09,190] Trial 6 finished with value: 0.5088973096010028 and parameters: {'var_threshold': 2.8, 'n_components': 3, 'n_units_0': 13}. Best is trial 5 with value: 0.5183728839384922.
# [I 2022-11-27 09:31:26,916] Trial 7 finished with value: 0.508950729040577 and parameters: {'var_threshold': 2.4, 'n_components': 3, 'n_units_0': 11}. Best is trial 5 with value: 0.5183728839384922.
# [I 2022-11-27 09:32:16,601] Trial 8 finished with value: 0.5198186087812416 and parameters: {'var_threshold': 4.0, 'n_components': 10, 'n_units_0': 12}. Best is trial 8 with value: 0.5198186087812416.
# [I 2022-11-27 09:33:46,053] Trial 9 finished with value: 0.5028169914271013 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 14, 'n_units_0': 19}. Best is trial 8 with value: 0.5198186087812416.
# [I 2022-11-27 09:35:19,678] Trial 10 finished with value: 0.5167531571447006 and parameters: {'var_threshold': 3.6, 'n_components': 13, 'n_units_0': 19}. Best is trial 8 with value: 0.5198186087812416.
# [I 2022-11-27 09:36:15,557] Trial 11 finished with value: 0.5231143480232746 and parameters: {'var_threshold': 2.8, 'n_components': 10, 'n_units_0': 13}. Best is trial 11 with value: 0.5231143480232746.
# [I 2022-11-27 09:37:07,587] Trial 12 finished with value: 0.5230841329082421 and parameters: {'var_threshold': 2.8, 'n_components': 10, 'n_units_0': 13}. Best is trial 11 with value: 0.5231143480232746.
# [I 2022-11-27 09:38:29,489] Trial 13 finished with value: 0.5118684227991295 and parameters: {'var_threshold': 2.8, 'n_components': 11, 'n_units_0': 17}. Best is trial 11 with value: 0.5231143480232746.
# [I 2022-11-27 09:39:21,571] Trial 14 finished with value: 0.507080230092009 and parameters: {'var_threshold': 3.0, 'n_components': 7, 'n_units_0': 14}. Best is trial 11 with value: 0.5231143480232746.
# [I 2022-11-27 09:40:36,359] Trial 15 finished with value: 0.5007224274455744 and parameters: {'var_threshold': 2.6, 'n_components': 12, 'n_units_0': 17}. Best is trial 11 with value: 0.5231143480232746.
# [I 2022-11-27 09:41:39,531] Trial 16 finished with value: 0.5184012804548368 and parameters: {'var_threshold': 2.0, 'n_components': 15, 'n_units_0': 13}. Best is trial 11 with value: 0.5231143480232746.
# [I 2022-11-27 09:42:42,899] Trial 17 finished with value: 0.5308139152054739 and parameters: {'var_threshold': 3.2, 'n_components': 9, 'n_units_0': 17}. Best is trial 17 with value: 0.5308139152054739.
# [I 2022-11-27 09:43:48,565] Trial 18 finished with value: 0.5170423544295282 and parameters: {'var_threshold': 3.2, 'n_components': 8, 'n_units_0': 16}. Best is trial 17 with value: 0.5308139152054739.
# [I 2022-11-27 09:43:54,514] Trial 19 finished with value: 0.504698772421732 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 1, 'n_units_0': 20}. Best is trial 17 with value: 0.5308139152054739.

Βλέπουμε πως και οι δύο παραπάνω εκτελέσεις, έδωσαν βέλτιστο αποτέλεσμα για `n_components = 9`. Γενικότερα, πολλές από τις εκτελέσεις συνέκλιναν κοντά στα 9 components. Το κρατάμε υπ'όψιν μας για τη συνέχεια.

##### Χωρίς Scaler:

In [ ]:
def nn_f1(trial):
    threshold = trial.suggest_float("var_threshold", 2., 4., step = 0.2)
    selector = VarianceThreshold(threshold = threshold)
    
    scaler = None
    ros = RandomOverSampler()

    n_components = trial.suggest_int('n_components', 1, 15)
    pca = PCA(n_components)
    
    layers= [trial.suggest_int(f'n_units', 10, 20)]

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)
    
    pipe = make_pipeline(scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring = 'f1')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_f1, n_trials = 20)

In [ ]:
# [I 2022-11-27 09:48:14,978] A new study created in memory with name: no-name-775a096e-286d-4e3b-8f8e-326ed2a694e6
# [I 2022-11-27 09:48:41,959] Trial 0 finished with value: 0.2981139312602158 and parameters: {'var_threshold': 3.0, 'n_components': 4, 'n_units': 14}. Best is trial 0 with value: 0.2981139312602158.
# [I 2022-11-27 09:49:40,986] Trial 1 finished with value: 0.5033216298005037 and parameters: {'var_threshold': 2.6, 'n_components': 15, 'n_units': 14}. Best is trial 1 with value: 0.5033216298005037.
# [I 2022-11-27 09:50:26,967] Trial 2 finished with value: 0.5106279600928544 and parameters: {'var_threshold': 2.8, 'n_components': 7, 'n_units': 17}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:51:33,307] Trial 3 finished with value: 0.4884493821769538 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 15, 'n_units': 17}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:51:47,510] Trial 4 finished with value: 0.30648626943536056 and parameters: {'var_threshold': 2.2, 'n_components': 2, 'n_units': 20}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:52:35,451] Trial 5 finished with value: 0.4999444092463895 and parameters: {'var_threshold': 3.8, 'n_components': 10, 'n_units': 12}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:53:26,824] Trial 6 finished with value: 0.5006644397176302 and parameters: {'var_threshold': 2.0, 'n_components': 8, 'n_units': 19}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:53:35,011] Trial 7 finished with value: 0.3066234282723699 and parameters: {'var_threshold': 3.8, 'n_components': 1, 'n_units': 18}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:54:38,797] Trial 8 finished with value: 0.5055563339992304 and parameters: {'var_threshold': 3.2, 'n_components': 15, 'n_units': 17}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:54:56,215] Trial 9 finished with value: 0.31068279995436476 and parameters: {'var_threshold': 2.4, 'n_components': 3, 'n_units': 11}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:55:38,514] Trial 10 finished with value: 0.4978751783992085 and parameters: {'var_threshold': 2.6, 'n_components': 7, 'n_units': 16}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:56:40,228] Trial 11 finished with value: 0.5003560527893675 and parameters: {'var_threshold': 3.2, 'n_components': 11, 'n_units': 16}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:57:20,891] Trial 12 finished with value: 0.5045926612241398 and parameters: {'var_threshold': 3.0,b 'n_components': 6, 'n_units': 18}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:58:13,095] Trial 13 finished with value: 0.4966532488535257 and parameters: {'var_threshold': 2.8, 'n_components': 12, 'n_units': 15}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 09:58:56,139] Trial 14 finished with value: 0.48769143012953425 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 5, 'n_units': 20}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 10:00:00,022] Trial 15 finished with value: 0.4942466128198676 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 13, 'n_units': 17}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 10:00:48,499] Trial 16 finished with value: 0.50311774346708 and parameters: {'var_threshold': 4.0, 'n_components': 9, 'n_units': 13}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 10:01:48,367] Trial 17 finished with value: 0.5099869160131913 and parameters: {'var_threshold': 2.8, 'n_components': 13, 'n_units': 16}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 10:02:35,058] Trial 18 finished with value: 0.5043451356565745 and parameters: {'var_threshold': 2.6, 'n_components': 13, 'n_units': 10}. Best is trial 2 with value: 0.5106279600928544.
# [I 2022-11-27 10:03:18,554] Trial 19 finished with value: 0.4985379400478001 and parameters: {'var_threshold': 2.8, 'n_components': 8, 'n_units': 15}. Best is trial 2 with value: 0.5106279600928544.

In [ ]:
# [I 2022-11-27 10:12:28,280] A new study created in memory with name: no-name-5e9f0857-2cca-4db3-a9bb-93cb50ccd580
# [I 2022-11-27 10:13:06,745] Trial 0 finished with value: 0.5047060135763325 and parameters: {'var_threshold': 2.2, 'n_components': 6, 'n_units': 16}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:13:14,752] Trial 1 finished with value: 0.3114894388170697 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 1, 'n_units': 15}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:14:10,351] Trial 2 finished with value: 0.4885672649097623 and parameters: {'var_threshold': 2.8, 'n_components': 12, 'n_units': 16}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:15:00,070] Trial 3 finished with value: 0.49627120514981177 and parameters: {'var_threshold': 3.8, 'n_components': 11, 'n_units': 13}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:15:08,198] Trial 4 finished with value: 0.3088904585846424 and parameters: {'var_threshold': 3.6, 'n_components': 1, 'n_units': 11}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:16:01,534] Trial 5 finished with value: 0.5041581573675796 and parameters: {'var_threshold': 3.8, 'n_components': 9, 'n_units': 17}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:16:31,090] Trial 6 finished with value: 0.30519260463945874 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 4, 'n_units': 11}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:17:22,956] Trial 7 finished with value: 0.4993898162664288 and parameters: {'var_threshold': 2.6, 'n_components': 10, 'n_units': 17}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:17:38,347] Trial 8 finished with value: 0.30764888139487384 and parameters: {'var_threshold': 2.0, 'n_components': 2, 'n_units': 14}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:18:22,150] Trial 9 finished with value: 0.5019535292573016 and parameters: {'var_threshold': 3.2, 'n_components': 13, 'n_units': 12}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:19:07,140] Trial 10 finished with value: 0.4930289039490692 and parameters: {'var_threshold': 2.0, 'n_components': 6, 'n_units': 19}. Best is trial 0 with value: 0.5047060135763325.
# [I 2022-11-27 10:19:56,461] Trial 11 finished with value: 0.5094935490996871 and parameters: {'var_threshold': 4.0, 'n_components': 7, 'n_units': 18}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:20:54,835] Trial 12 finished with value: 0.4891706692419677 and parameters: {'var_threshold': 2.4, 'n_components': 7, 'n_units': 20}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:21:32,432] Trial 13 finished with value: 0.5047478985051777 and parameters: {'var_threshold': 4.0, 'n_components': 5, 'n_units': 18}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:22:50,101] Trial 14 finished with value: 0.48828621264912747 and parameters: {'var_threshold': 4.0, 'n_components': 15, 'n_units': 19}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:23:25,083] Trial 15 finished with value: 0.30693853421213124 and parameters: {'var_threshold': 4.0, 'n_components': 4, 'n_units': 18}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:24:05,113] Trial 16 finished with value: 0.3130783756253471 and parameters: {'var_threshold': 3.0, 'n_components': 4, 'n_units': 20}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:24:55,794] Trial 17 finished with value: 0.49607319973301234 and parameters: {'var_threshold': 3.6, 'n_components': 8, 'n_units': 18}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:25:32,595] Trial 18 finished with value: 0.5020312791625887 and parameters: {'var_threshold': 4.0, 'n_components': 5, 'n_units': 15}. Best is trial 11 with value: 0.5094935490996871.
# [I 2022-11-27 10:26:22,284] Trial 19 finished with value: 0.49372393222150784 and parameters: {'var_threshold': 3.6, 'n_components': 8, 'n_units': 18}. Best is trial 11 with value: 0.5094935490996871.

Σε γενικές γραμμές βλέπουμε πως η έλλειψη του scaler, μειώνει όχι υπερβολικά, αλλά αισθητά την απόδοση του ταξινομητή μας, οπότε επιλέγουμε να τον κρατήσουμε.

##### Χωρίς sampler:

Θα ελέγξουμε αν έχει νόημα να αφαιρέσουμε τον sampler (που διαισθητικά λόγω του imbalance, αναμένουμε πως θα πρέπει να τον κρατήσουμε).

In [ ]:
def nn_f1(trial):
    threshold = trial.suggest_float("var_threshold", 2., 4., step = 0.2)
    selector = VarianceThreshold(threshold = threshold)
    
    scaler = StandardScaler()
    ros = None

    n_components = trial.suggest_int('n_components', 1, 15)
    pca = PCA(n_components)
    
    layers= [trial.suggest_int(f'n_units', 10, 20)]

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)
    
    pipe = make_pipeline(selector, scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring = 'f1')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_f1, n_trials = 20)

In [ ]:
# [I 2022-11-27 10:47:42,828] A new study created in memory with name: no-name-a2ba033d-a107-4df9-a644-5197d987d26d
# [I 2022-11-27 10:47:57,028] Trial 0 finished with value: 0.36287956535159205 and parameters: {'var_threshold': 2.4, 'n_components': 3, 'n_units': 16}. Best is trial 0 with value: 0.36287956535159205.
# [I 2022-11-27 10:48:02,663] Trial 1 finished with value: 0.3158512545080055 and parameters: {'var_threshold': 4.0, 'n_components': 1, 'n_units': 19}. Best is trial 0 with value: 0.36287956535159205.
# [I 2022-11-27 10:48:35,723] Trial 2 finished with value: 0.35533048725443994 and parameters: {'var_threshold': 4.0, 'n_components': 10, 'n_units': 18}. Best is trial 0 with value: 0.36287956535159205.
# [I 2022-11-27 10:49:17,119] Trial 3 finished with value: 0.4566356908420423 and parameters: {'var_threshold': 3.6, 'n_components': 15, 'n_units': 15}. Best is trial 3 with value: 0.4566356908420423.
# [I 2022-11-27 10:49:42,349] Trial 4 finished with value: 0.44936820860958065 and parameters: {'var_threshold': 3.0, 'n_components': 12, 'n_units': 13}. Best is trial 3 with value: 0.4566356908420423.
# [I 2022-11-27 10:50:33,378] Trial 5 finished with value: 0.4638361866544759 and parameters: {'var_threshold': 3.0, 'n_components': 13, 'n_units': 20}. Best is trial 5 with value: 0.4638361866544759.
# [I 2022-11-27 10:50:43,637] Trial 6 finished with value: 0.4182745250168904 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 3, 'n_units': 13}. Best is trial 5 with value: 0.4638361866544759.
# [I 2022-11-27 10:50:58,942] Trial 7 finished with value: 0.44342901466924395 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 12}. Best is trial 5 with value: 0.4638361866544759.
# [I 2022-11-27 10:51:12,806] Trial 8 finished with value: 0.4224806682211347 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 5, 'n_units': 16}. Best is trial 5 with value: 0.4638361866544759.
# [I 2022-11-27 10:51:18,509] Trial 9 finished with value: 0.3831030076995776 and parameters: {'var_threshold': 3.6, 'n_components': 1, 'n_units': 18}. Best is trial 5 with value: 0.4638361866544759.
# [I 2022-11-27 10:51:42,894] Trial 10 finished with value: 0.4608387950808014 and parameters: {'var_threshold': 2.0, 'n_components': 14, 'n_units': 10}. Best is trial 5 with value: 0.4638361866544759.
# [I 2022-11-27 10:52:08,316] Trial 11 finished with value: 0.4678071218818226 and parameters: {'var_threshold': 2.0, 'n_components': 14, 'n_units': 10}. Best is trial 11 with value: 0.4678071218818226.
# [I 2022-11-27 10:52:51,072] Trial 12 finished with value: 0.45262201986376593 and parameters: {'var_threshold': 2.6, 'n_components': 12, 'n_units': 20}. Best is trial 11 with value: 0.4678071218818226.
# [I 2022-11-27 10:53:12,012] Trial 13 finished with value: 0.44844087586172704 and parameters: {'var_threshold': 2.0, 'n_components': 10, 'n_units': 10}. Best is trial 11 with value: 0.4678071218818226.
# [I 2022-11-27 10:53:44,643] Trial 14 finished with value: 0.45009031907385666 and parameters: {'var_threshold': 2.6, 'n_components': 12, 'n_units': 15}. Best is trial 11 with value: 0.4678071218818226.
# [I 2022-11-27 10:54:17,004] Trial 15 finished with value: 0.47329902770316723 and parameters: {'var_threshold': 2.8, 'n_components': 15, 'n_units': 12}. Best is trial 15 with value: 0.47329902770316723.
# [I 2022-11-27 10:54:47,968] Trial 16 finished with value: 0.48497426741757454 and parameters: {'var_threshold': 2.4, 'n_components': 15, 'n_units': 11}. Best is trial 16 with value: 0.48497426741757454.
# [I 2022-11-27 10:55:08,219] Trial 17 finished with value: 0.4179415349027845 and parameters: {'var_threshold': 2.4, 'n_components': 9, 'n_units': 12}. Best is trial 16 with value: 0.48497426741757454.
# [I 2022-11-27 10:55:40,380] Trial 18 finished with value: 0.4909934970131206 and parameters: {'var_threshold': 2.6, 'n_components': 15, 'n_units': 12}. Best is trial 18 with value: 0.4909934970131206.
# [I 2022-11-27 10:55:58,706] Trial 19 finished with value: 0.42353753213523565 and parameters: {'var_threshold': 2.4, 'n_components': 7, 'n_units': 11}. Best is trial 18 with value: 0.4909934970131206.

Βλέπουμε πως έχουμε σημαντική μείωση στην απόδοση μετά από αφαίρεση του sampler, οπότε και τον κρατάμε.

##### Τελικές δοκιμές με πλήρες pipeline

Τέλος, θα κάνουμε μία πιο απαιτητική δοκιμή, ώστε να καταλήξουμε στο pipeline που θα θεωρήσουμε "βέλτιστο". Δοκιμαζουμε το full pipeline, με thresholds στο [2, 4] και n_components στο {6, .., 12}.

In [ ]:
def nn_f1(trial):
    threshold = trial.suggest_float("var_threshold", 2., 4., step = 0.2)
    selector = VarianceThreshold(threshold = threshold)
    
    scaler = StandardScaler()
    ros = RandomOverSampler()

    n_components = trial.suggest_int('n_components', 6, 13)
    pca = PCA(n_components)
    
    layers= [trial.suggest_int(f'n_units', 10, 20)]

    clf = MLPClassifier(hidden_layer_sizes = tuple(layers), max_iter = 1000)
    
    pipe = make_pipeline(selector, scaler, ros, pca, clf)

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring = 'f1')
    return score.mean()

study = optuna.create_study(direction = 'maximize')
study.optimize(nn_f1, n_trials = 100)

In [ ]:
# [I 2022-11-27 16:39:27,693] A new study created in memory with name: no-name-112569a9-92a0-436b-95cd-ce5a8f3d994d
# [I 2022-11-27 16:40:16,200] Trial 0 finished with value: 0.5146390587392552 and parameters: {'var_threshold': 2.8, 'n_components': 6, 'n_units': 14}. Best is trial 0 with value: 0.5146390587392552.
# [I 2022-11-27 16:41:11,727] Trial 1 finished with value: 0.5111783965393549 and parameters: {'var_threshold': 3.0, 'n_components': 11, 'n_units': 12}. Best is trial 0 with value: 0.5146390587392552.
# [I 2022-11-27 16:42:48,263] Trial 2 finished with value: 0.5145414150183577 and parameters: {'var_threshold': 2.0, 'n_components': 13, 'n_units': 19}. Best is trial 0 with value: 0.5146390587392552.
# [I 2022-11-27 16:44:26,615] Trial 3 finished with value: 0.5008533057801416 and parameters: {'var_threshold': 2.2, 'n_components': 13, 'n_units': 20}. Best is trial 0 with value: 0.5146390587392552.
# [I 2022-11-27 16:45:14,352] Trial 4 finished with value: 0.5113071790351977 and parameters: {'var_threshold': 2.8, 'n_components': 6, 'n_units': 14}. Best is trial 0 with value: 0.5146390587392552.
# [I 2022-11-27 16:46:43,570] Trial 5 finished with value: 0.517341385281124 and parameters: {'var_threshold': 2.6, 'n_components': 12, 'n_units': 17}. Best is trial 5 with value: 0.517341385281124.
# [I 2022-11-27 16:48:00,317] Trial 6 finished with value: 0.5369788848726167 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 20}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:49:20,544] Trial 7 finished with value: 0.5070203717984271 and parameters: {'var_threshold': 3.0, 'n_components': 10, 'n_units': 16}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:50:07,430] Trial 8 finished with value: 0.506045549660499 and parameters: {'var_threshold': 2.4, 'n_components': 11, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:51:35,230] Trial 9 finished with value: 0.45648322547513215 and parameters: {'var_threshold': 3.8, 'n_components': 11, 'n_units': 19}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:52:46,238] Trial 10 finished with value: 0.45227851297473765 and parameters: {'var_threshold': 3.8, 'n_components': 8, 'n_units': 17}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:53:49,440] Trial 11 finished with value: 0.5136083367065353 and parameters: {'var_threshold': 3.6, 'n_components': 8, 'n_units': 17}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:55:00,857] Trial 12 finished with value: 0.5090318618757481 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 8, 'n_units': 18}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:56:20,081] Trial 13 finished with value: 0.501043660675137 and parameters: {'var_threshold': 2.6, 'n_components': 9, 'n_units': 20}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:57:13,564] Trial 14 finished with value: 0.5206816878728843 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 7, 'n_units': 15}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:58:01,916] Trial 15 finished with value: 0.5161910119133274 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 7, 'n_units': 14}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:58:50,808] Trial 16 finished with value: 0.519583980489118 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 7, 'n_units': 15}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 16:59:33,429] Trial 17 finished with value: 0.4640379922382893 and parameters: {'var_threshold': 4.0, 'n_components': 7, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:00:45,081] Trial 18 finished with value: 0.5247783541948629 and parameters: {'var_threshold': 3.2, 'n_components': 9, 'n_units': 15}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:01:22,455] Trial 19 finished with value: 0.5224711700546268 and parameters: {'var_threshold': 3.2, 'n_components': 9, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:02:18,265] Trial 20 finished with value: 0.5290740023910824 and parameters: {'var_threshold': 3.2, 'n_components': 10, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:03:24,858] Trial 21 finished with value: 0.4999267302233331 and parameters: {'var_threshold': 3.0, 'n_components': 10, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:04:20,994] Trial 22 finished with value: 0.5260583381357128 and parameters: {'var_threshold': 3.2, 'n_components': 9, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:05:18,752] Trial 23 finished with value: 0.5246390367490583 and parameters: {'var_threshold': 3.2, 'n_components': 10, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:06:07,960] Trial 24 finished with value: 0.5160027351223444 and parameters: {'var_threshold': 3.6, 'n_components': 8, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:06:44,775] Trial 25 finished with value: 0.5017220179027554 and parameters: {'var_threshold': 2.8, 'n_components': 9, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:07:51,249] Trial 26 finished with value: 0.5219723374180877 and parameters: {'var_threshold': 3.2, 'n_components': 12, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:08:35,385] Trial 27 finished with value: 0.5143443025405159 and parameters: {'var_threshold': 3.6, 'n_components': 10, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:09:31,741] Trial 28 finished with value: 0.5090015544490629 and parameters: {'var_threshold': 2.6, 'n_components': 6, 'n_units': 16}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:10:23,212] Trial 29 finished with value: 0.5048065656577947 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 14}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:11:19,650] Trial 30 finished with value: 0.5260772430802166 and parameters: {'var_threshold': 2.8, 'n_components': 12, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:12:26,298] Trial 31 finished with value: 0.5100996086331725 and parameters: {'var_threshold': 2.8, 'n_components': 12, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:13:12,435] Trial 32 finished with value: 0.5136689350851137 and parameters: {'var_threshold': 3.0, 'n_components': 11, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:14:09,374] Trial 33 finished with value: 0.5210826547329777 and parameters: {'var_threshold': 3.2, 'n_components': 12, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:15:02,513] Trial 34 finished with value: 0.5260942226824477 and parameters: {'var_threshold': 2.4, 'n_components': 13, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:16:12,858] Trial 35 finished with value: 0.5173242994203988 and parameters: {'var_threshold': 2.0, 'n_components': 13, 'n_units': 14}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:17:11,228] Trial 36 finished with value: 0.5119413419274954 and parameters: {'var_threshold': 2.4, 'n_components': 13, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:18:31,660] Trial 37 finished with value: 0.5016178200900063 and parameters: {'var_threshold': 2.2, 'n_components': 13, 'n_units': 16}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:19:36,724] Trial 38 finished with value: 0.5080019048789763 and parameters: {'var_threshold': 2.4, 'n_components': 11, 'n_units': 14}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:21:07,395] Trial 39 finished with value: 0.5127444132353879 and parameters: {'var_threshold': 2.2, 'n_components': 12, 'n_units': 19}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:22:24,147] Trial 40 finished with value: 0.5155372231446866 and parameters: {'var_threshold': 2.6, 'n_components': 13, 'n_units': 15}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:23:15,956] Trial 41 finished with value: 0.5049346549343436 and parameters: {'var_threshold': 2.8, 'n_components': 11, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:24:07,253] Trial 42 finished with value: 0.517318350240634 and parameters: {'var_threshold': 3.0, 'n_components': 12, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:25:04,066] Trial 43 finished with value: 0.5083846416168447 and parameters: {'var_threshold': 2.8, 'n_components': 10, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:26:14,331] Trial 44 finished with value: 0.5184240728893676 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 8, 'n_units': 18}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:27:16,128] Trial 45 finished with value: 0.5291774938410446 and parameters: {'var_threshold': 3.2, 'n_components': 12, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:28:17,805] Trial 46 finished with value: 0.5063955789106181 and parameters: {'var_threshold': 3.0, 'n_components': 13, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:29:12,752] Trial 47 finished with value: 0.5155246084779639 and parameters: {'var_threshold': 2.4, 'n_components': 12, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:30:52,676] Trial 48 finished with value: 0.4897327737359266 and parameters: {'var_threshold': 2.6, 'n_components': 11, 'n_units': 20}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:32:05,451] Trial 49 finished with value: 0.5134316179496358 and parameters: {'var_threshold': 3.6, 'n_components': 13, 'n_units': 14}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:32:58,706] Trial 50 finished with value: 0.5032546432583389 and parameters: {'var_threshold': 2.0, 'n_components': 12, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:33:44,281] Trial 51 finished with value: 0.5338632215181891 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:34:30,486] Trial 52 finished with value: 0.5228991925596234 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 6, 'n_units': 14}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:35:15,340] Trial 53 finished with value: 0.5274351453125155 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 13}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:35:49,309] Trial 54 finished with value: 0.5268594142955904 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:36:30,517] Trial 55 finished with value: 0.5290181774447794 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:36:56,784] Trial 56 finished with value: 0.5205549127963671 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 7, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:37:34,017] Trial 57 finished with value: 0.5328871029279542 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:38:14,544] Trial 58 finished with value: 0.5236092914174225 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 8, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:38:43,429] Trial 59 finished with value: 0.51028437179976 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:39:25,721] Trial 60 finished with value: 0.5318526158324002 and parameters: {'var_threshold': 3.2, 'n_components': 8, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:40:08,710] Trial 61 finished with value: 0.5315964302756981 and parameters: {'var_threshold': 3.2, 'n_components': 8, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:40:46,472] Trial 62 finished with value: 0.5277396572147275 and parameters: {'var_threshold': 3.2, 'n_components': 8, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:41:26,324] Trial 63 finished with value: 0.5208876817766708 and parameters: {'var_threshold': 3.6, 'n_components': 7, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:42:10,030] Trial 64 finished with value: 0.529267688851271 and parameters: {'var_threshold': 3.2, 'n_components': 8, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:42:50,323] Trial 65 finished with value: 0.5120326192379168 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 8, 'n_units': 12}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:43:32,505] Trial 66 finished with value: 0.5066426785830942 and parameters: {'var_threshold': 3.0, 'n_components': 8, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:44:08,122] Trial 67 finished with value: 0.5300695444251107 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:44:38,063] Trial 68 finished with value: 0.5092192472086806 and parameters: {'var_threshold': 3.0, 'n_components': 7, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:45:12,630] Trial 69 finished with value: 0.5240132498140203 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 8, 'n_units': 10}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:45:47,085] Trial 70 finished with value: 0.5323247790761512 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:46:20,239] Trial 71 finished with value: 0.5313707941760704 and parameters: {'var_threshold': 3.2, 'n_components': 7, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:46:55,217] Trial 72 finished with value: 0.5230315032185496 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 7, 'n_units': 11}. Best is trial 6 with value: 0.5369788848726167.
# [I 2022-11-27 17:47:29,596] Trial 73 finished with value: 0.5382298595369841 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:48:06,784] Trial 74 finished with value: 0.5363197839969829 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 11}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:48:35,308] Trial 75 finished with value: 0.5114036820183472 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:49:13,758] Trial 76 finished with value: 0.5061257436513409 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 11}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:49:47,842] Trial 77 finished with value: 0.5351294617858096 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:50:21,220] Trial 78 finished with value: 0.5233366374710788 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:50:50,040] Trial 79 finished with value: 0.5341264828291797 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:51:52,163] Trial 80 finished with value: 0.506369979647939 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 18}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:52:23,140] Trial 81 finished with value: 0.5283372455729219 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:52:52,487] Trial 82 finished with value: 0.5348741888068936 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:53:24,832] Trial 83 finished with value: 0.5221691872708343 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:53:58,691] Trial 84 finished with value: 0.534218948480987 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:54:26,113] Trial 85 finished with value: 0.531309334344207 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:54:54,920] Trial 86 finished with value: 0.5216533027870773 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:55:23,139] Trial 87 finished with value: 0.5110967319977682 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:55:52,617] Trial 88 finished with value: 0.5340393734334634 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:56:24,511] Trial 89 finished with value: 0.5219772760433496 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:56:49,596] Trial 90 finished with value: 0.5278332405651789 and parameters: {'var_threshold': 3.6, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:57:26,419] Trial 91 finished with value: 0.5353165144866814 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 73 with value: 0.5382298595369841.
# [I 2022-11-27 17:57:58,833] Trial 92 finished with value: 0.5439340360470697 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 17:58:31,915] Trial 93 finished with value: 0.5163227678659154 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 10}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 17:59:02,227] Trial 94 finished with value: 0.5346395843960715 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 18:00:03,451] Trial 95 finished with value: 0.5298560867391383 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 17}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 18:01:11,324] Trial 96 finished with value: 0.5077419218845108 and parameters: {'var_threshold': 3.0, 'n_components': 6, 'n_units': 20}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 18:01:47,456] Trial 97 finished with value: 0.5370431856274395 and parameters: {'var_threshold': 3.2, 'n_components': 6, 'n_units': 10}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 18:02:19,000] Trial 98 finished with value: 0.519350247968812 and parameters: {'var_threshold': 3.4000000000000004, 'n_components': 6, 'n_units': 10}. Best is trial 92 with value: 0.5439340360470697.
# [I 2022-11-27 18:03:15,573] Trial 99 finished with value: 0.47070533285591126 and parameters: {'var_threshold': 4.0, 'n_components': 6, 'n_units': 19}. Best is trial 92 with value: 0.5439340360470697.

Βλέπουμε πως οι περισσότερες εκτελέσεις συγκλίνουν σε 6 n_components. Επίσης συγκλίνουν σε 10 νευρώνες. Το variance threshold έχει κάποια διακύμανση, αλλά φαίνεται να συγκλίνει στο 3.2. Κάνουμε σύντομο GridSearch γύρω από το 3.2 ώστε να καταλήξουμε στο τελικό μοντέλο.

In [ ]:
selector = VarianceThreshold()
scaler = StandardScaler()
ros = RandomOverSampler()
pca = PCA(n_components = 6)
clf = MLPClassifier(hidden_layer_sizes = (10, ), max_iter = 1000)

In [ ]:
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('mlp', clf)], memory = 'tmp')

In [ ]:
vthreshold = np.linspace(2.8, 3.4, 7, endpoint = True)

estimator = GridSearchCV(pipe, dict(selector__threshold = vthreshold), 
                         cv = 10, scoring = 'f1', n_jobs = -1).fit(X_train, y_train)
print(f"Best Parameters: {estimator.best_params_}")
print(f"Best Score: {100 * estimator.best_score_:.3f}%")

Καταλήγουμε σε αρχιτεκτονική με variance threshold = 3.3, standard scaler, random oversampler, PCA 6 συνιστωσών και MLP με 1 hidden layer & 10 νευρώνες. (το mean f1-score ενδεχομένως να μην είναι "συνεπές" κατά την εκπαίδευση ακόμη και με ίδιες παραμέτρους, λόγω της τυχαιότητας που εισάγει ο ros και η εκπαίδευση του MLP)

In [ ]:
# Our choice for 'maximizing f1'
print(f"MLP: Best Parameters: {estimator.best_estimator_}")
print(f"     Training time {1000 * (estimator.refit_time_):.5f} ms")
start = time.time()
mlp_f1_preds = estimator.predict(X_test)
stop = time.time()
print(f"     Classification time {1000 * (stop - start):.5f} ms")
print(f"     F1 Score {100 * f1_score(y_test, mlp_f1_preds):.3f}%")

Στην ootb υλοποίηση το πρώτο επίπεδο έχει 100 νευρώνες, κάτι που στις παραπάνω δοκιμές ρίξαμε σημαντικά. Αυτό επιταχύνει την εκπαίδευση και σε συνδυασμό με το υπόλοιπο pipeline, μας έδωσε και αρκετά ικανοποιητικό αποτέλεσμα.

#### SVM Optimization

Αρχικά, έγινε δοκιμή με την εισαγωγή selector σαν 1ο βήμα, αλλά η επίδοση ήταν χαμηλή. Για αυτόν τον λόγο, δεν το συμπεριλαμβάνουμε στις δοκιμές μας. Έχουμε ήδη κάνει προσεκτική αφαίρεση χαρακτηριστικών, οπότε δεν μας παραξενεύει πως ο selector δεν βελτίωσε την επίδοση των ταξινομητών μας.

In [ ]:
def svm(trial, scoring):
    if scoring != 'accuracy' and scoring != 'f1':
        print("UNKNOWN SCORING")
        return
    
    #var_threshold = trial.suggest_float("var_threshold", 0, 0.3, step = 0.05)
    scalers = trial.suggest_categorical("scalers", ['standard', 'no_scaler'])
    samplers = trial.suggest_categorical("samplers", ['ros', 'no_sampler'])
    n_components = trial.suggest_int('n_components', 5, 25)
    
    #selector = VarianceThreshold(threshold = var_threshold)
    scaler = StandardScaler() if scalers == 'standard' else None    # Create another objective with no scaler?
    sampler = RandomOverSampler() if samplers == 'ros' else None    # << << <<
    pca = PCA(n_components)
    
    C = trial.suggest_float('C', 0.001, 1000., log = True)
    kernel = trial.suggest_categorical('kernel', ['rbf', 'linear', 'poly'])
    
    clf = None
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 6)
        clf = SVC(C = C, kernel = kernel, degree = degree)
    elif kernel == 'rbf':
        gamma = trial.suggest_float('gamma', 0.001, 1000., log = True)
        clf = SVC(C = C, kernel = kernel, gamma = gamma)
    else: # kernel == 'linear'
        clf = SVC(C = C, kernel = kernel)
    
    pipe = make_pipeline(scaler, sampler, pca, clf) #no selector, because bad performance

    score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring=scoring)
    return score.mean()


def svm_accuracy(trial):
    return svm(trial, 'accuracy')

def svm_f1(trial):
    return svm(trial, 'f1')

##### Accuracy


In [ ]:
optuna.create_study(direction = 'maximize').optimize(svm_accuracy, n_trials = 10)

In [ ]:
### 1η εκτέλεση ###
# [I 2022-12-01 22:56:46,080] Trial 0 finished with value: 0.8329800786014365 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 10, 'C': 10.234078949592195, 'kernel': 'rbf', 'gamma': 49.483886818055154}. Best is trial 0 with value: 0.8329800786014365.
# [I 2022-12-01 22:57:09,215] Trial 1 finished with value: 0.7677463070876812 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 19, 'C': 0.14443081509097871, 'kernel': 'linear'}. Best is trial 0 with value: 0.8329800786014365.
# [I 2022-12-01 22:57:30,672] Trial 2 finished with value: 0.7845304241767178 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 18, 'C': 4.217316998082073, 'kernel': 'rbf', 'gamma': 0.012227599071576343}. Best is trial 0 with value: 0.8329800786014365.
# [I 2022-12-01 22:58:07,944] Trial 3 finished with value: 0.8329800786014365 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 24, 'C': 0.030851606120178753, 'kernel': 'rbf', 'gamma': 3.4001225773286037}. Best is trial 0 with value: 0.8329800786014365.
# [I 2022-12-01 22:58:28,353] Trial 4 finished with value: 0.733313908840403 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 6, 'C': 0.05641492420339519, 'kernel': 'poly', 'degree': 3}. Best is trial 0 with value: 0.8329800786014365.
# [I 2022-12-01 22:58:53,477] Trial 5 finished with value: 0.8295717576907441 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 24, 'C': 8.358364468981815, 'kernel': 'poly', 'degree': 5}. Best is trial 0 with value: 0.8329800786014365.
# [I 2022-12-01 22:58:59,793] Trial 6 finished with value: 0.8606021592808419 and parameters: {'scalers': 'standard', 'samplers': 'no_sampler', 'n_components': 17, 'C': 6.73169132371167, 'kernel': 'rbf', 'gamma': 0.019912744243972137}. Best is trial 6 with value: 0.8606021592808419.
# [I 2022-12-01 22:59:56,658] Trial 7 finished with value: 0.8170452184126124 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 22, 'C': 529.1011355661387, 'kernel': 'rbf', 'gamma': 0.023773739809182313}. Best is trial 6 with value: 0.8606021592808419.
# [I 2022-12-01 23:02:21,940] Trial 8 finished with value: 0.5939047748114017 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 17, 'C': 40.63980949453461, 'kernel': 'poly', 'degree': 2}. Best is trial 6 with value: 0.8606021592808419.
# [I 2022-12-01 23:02:48,285] Trial 9 finished with value: 0.8329800786014365 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 20, 'C': 740.5731341690438, 'kernel': 'rbf', 'gamma': 0.27392145307164445}. Best is trial 6 with value: 0.8606021592808419.

In [ ]:
### 2η εκτέλεση ###
# [I 2022-12-01 23:24:30,354] Trial 0 finished with value: 0.45540181596422286 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 22, 'C': 0.03916449937360525, 'kernel': 'poly', 'degree': 2}. Best is trial 0 with value: 0.45540181596422286.
# [I 2022-12-01 23:26:30,427] Trial 1 finished with value: 0.8089583051000588 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 19, 'C': 537.8735214267213, 'kernel': 'rbf', 'gamma': 0.008071996408663355}. Best is trial 1 with value: 0.8089583051000588.
# [I 2022-12-01 23:26:55,420] Trial 2 finished with value: 0.8110859646745269 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 10, 'C': 0.011397685362100377, 'kernel': 'poly', 'degree': 5}. Best is trial 2 with value: 0.8110859646745269.
# [I 2022-12-01 23:27:24,891] Trial 3 finished with value: 0.8336233455301081 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 15, 'C': 0.003119517321087387, 'kernel': 'poly', 'degree': 5}. Best is trial 3 with value: 0.8336233455301081.
# [I 2022-12-01 23:29:32,806] Trial 4 finished with value: 0.8200198762253242 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 17, 'C': 122.94697648695848, 'kernel': 'poly', 'degree': 3}. Best is trial 3 with value: 0.8336233455301081.
# [I 2022-12-01 23:30:01,229] Trial 5 finished with value: 0.7987645119031488 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 9, 'C': 0.09973248467446597, 'kernel': 'rbf', 'gamma': 0.5030820635297912}. Best is trial 3 with value: 0.8336233455301081.
# [I 2022-12-01 23:30:27,795] Trial 6 finished with value: 0.5038555359804852 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 11, 'C': 0.02874726978685776, 'kernel': 'poly', 'degree': 6}. Best is trial 3 with value: 0.8336233455301081.
# [I 2022-12-01 23:30:45,481] Trial 7 finished with value: 0.7677431449609251 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 20, 'C': 0.01432837874179172, 'kernel': 'linear'}. Best is trial 3 with value: 0.8336233455301081.
# [I 2022-12-01 23:31:10,537] Trial 8 finished with value: 0.4640863712336811 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 10, 'C': 0.0630601500592455, 'kernel': 'poly', 'degree': 2}. Best is trial 3 with value: 0.8336233455301081.
# [I 2022-12-01 23:31:27,230] Trial 9 finished with value: 0.7513786872656639 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 13, 'C': 0.0018376929497802862, 'kernel': 'linear'}. Best is trial 3 with value: 0.8336233455301081.

Τελικά, **ο καλύτερος συνδυασμός για το accuracy**, ίσο με 0.861 (από την 1η εκτέλεση), είναι: <br>
`{'scalers': 'standard', 'samplers': 'no_sampler', 'n_components': 17, 'C': 6.73169132371167, 'kernel': 'rbf', 'gamma': 0.019912744243972137}`

In [ ]:
scaler = StandardScaler()
sampler = None
pca = PCA(17)
clf = SVC(C = 6.73169132371167, kernel = 'rbf', gamma = 0.019912744243972137)

pipe_acc = make_pipeline(scaler, sampler, pca, clf)


start_time = time.time()
pipe_acc.fit(X_train, y_train)
print(f"Time spent to fit: {time.time() - start_time:.6f} seconds")  

start_time = time.time()  
svm_acc_preds = pipe_acc.predict(X_test)
print(f"Time spent to test: {time.time() - start_time:.6f} seconds")

print(f"Accuracy on test set: {100 * accuracy_score(y_test, svm_acc_preds):.3f}%")

##### F1-Score

In [ ]:
optuna.create_study(direction = 'maximize').optimize(svm_f1, n_trials = 10)

In [ ]:
### 1η εκτέλεση ###
# [I 2022-12-01 23:35:53,843] Trial 0 finished with value: 0.3273251529233861 and parameters: {'scalers': 'standard', 'samplers': 'no_sampler', 'n_components': 17, 'C': 9.229604762298397, 'kernel': 'poly', 'degree': 4}. Best is trial 0 with value: 0.3273251529233861.
# [I 2022-12-01 23:37:26,699] Trial 1 finished with value: 0.3554686242784295 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 25, 'C': 2.838657438791962, 'kernel': 'linear'}. Best is trial 1 with value: 0.3554686242784295.
# [I 2022-12-01 23:37:38,648] Trial 2 finished with value: 0.0 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 10, 'C': 0.4166596836441067, 'kernel': 'rbf', 'gamma': 314.63473929895156}. Best is trial 1 with value: 0.3554686242784295.
# [I 2022-12-01 23:38:17,220] Trial 3 finished with value: 0.5258437371052398 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 24, 'C': 0.003921305905693351, 'kernel': 'rbf', 'gamma': 0.0024556906883993006}. Best is trial 3 with value: 0.5258437371052398.
# [I 2022-12-01 23:38:51,461] Trial 4 finished with value: 0.5068430296175065 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 10, 'C': 32.221815114159426, 'kernel': 'rbf', 'gamma': 0.04115593568011447}. Best is trial 3 with value: 0.5258437371052398.
# [I 2022-12-01 23:39:38,733] Trial 5 finished with value: 0.30304941579751077 and parameters: {'scalers': 'standard', 'samplers': 'no_sampler', 'n_components': 12, 'C': 68.28530847870502, 'kernel': 'poly', 'degree': 4}. Best is trial 3 with value: 0.5258437371052398.
# [I 2022-12-01 23:39:42,533] Trial 6 finished with value: 0.018661257606490872 and parameters: {'scalers': 'standard', 'samplers': 'no_sampler', 'n_components': 11, 'C': 0.0052062886887551466, 'kernel': 'linear'}. Best is trial 3 with value: 0.5258437371052398.
# [I 2022-12-01 23:39:49,328] Trial 7 finished with value: 0.41645535306886244 and parameters: {'scalers': 'standard', 'samplers': 'no_sampler', 'n_components': 19, 'C': 8.050055606908554, 'kernel': 'rbf', 'gamma': 0.009528814230329882}. Best is trial 3 with value: 0.5258437371052398.
# [I 2022-12-01 23:40:07,026] Trial 8 finished with value: 0.5383067263587992 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 22, 'C': 0.008347696726773763, 'kernel': 'linear'}. Best is trial 8 with value: 0.5383067263587992.

In [ ]:
### 2η εκτέλεση ###
# [I 2022-12-02 00:05:16,845] Trial 0 finished with value: 0.4398023056333328 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 17, 'C': 5.420070009851526, 'kernel': 'poly', 'degree': 4}. Best is trial 0 with value: 0.4398023056333328.
# [I 2022-12-02 00:05:39,870] Trial 1 finished with value: 0.031862337805592146 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 18, 'C': 89.87509203191529, 'kernel': 'rbf', 'gamma': 0.07524251417369043}. Best is trial 0 with value: 0.4398023056333328.
# [I 2022-12-02 00:06:03,799] Trial 2 finished with value: 0.47473645777120466 and parameters: {'scalers': 'standard', 'samplers': 'ros', 'n_components': 23, 'C': 33.69512395430203, 'kernel': 'poly', 'degree': 5}. Best is trial 2 with value: 0.47473645777120466.
# [I 2022-12-02 00:06:52,905] Trial 3 finished with value: 0.4723598674766015 and parameters: {'scalers': 'no_scaler', 'samplers': 'ros', 'n_components': 11, 'C': 6.734154670016111, 'kernel': 'poly', 'degree': 3}. Best is trial 2 with value: 0.47473645777120466.
# [I 2022-12-02 00:06:57,754] Trial 4 finished with value: 0.0 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 11, 'C': 0.013490743993834998, 'kernel': 'linear'}. Best is trial 2 with value: 0.47473645777120466.
# [I 2022-12-02 00:07:07,459] Trial 5 finished with value: 0.19981848056826762 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 15, 'C': 1.9943626842987117, 'kernel': 'poly', 'degree': 4}. Best is trial 2 with value: 0.47473645777120466.
# [I 2022-12-02 00:07:29,362] Trial 6 finished with value: 0.32998389577235515 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 19, 'C': 324.41077489098257, 'kernel': 'poly', 'degree': 6}. Best is trial 2 with value: 0.47473645777120466.
# [I 2022-12-02 00:07:47,736] Trial 7 finished with value: 0.0 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 10, 'C': 0.6531100361044078, 'kernel': 'linear'}. Best is trial 2 with value: 0.47473645777120466.
# [I 2022-12-02 00:07:53,574] Trial 8 finished with value: 0.03228780030416338 and parameters: {'scalers': 'no_scaler', 'samplers': 'no_sampler', 'n_components': 12, 'C': 0.030848567746603973, 'kernel': 'poly', 'degree': 4}. Best is trial 2 with value: 0.47473645777120466.

Τελικά, **ο καλύτερος συνδυασμός για το F1-Score**, ίσο με 0.538 (από την 1η εκτέλεση), είναι: <br>
`{'scalers': 'standard', 'samplers': 'ros', 'n_components': 22, 'C': 0.008347696726773763, 'kernel': 'linear'}`

In [ ]:
scaler = StandardScaler()
sampler = RandomOverSampler()
pca = PCA(22)
clf = SVC(C = 0.008347696726773763, kernel = 'linear')

pipe_f1 = make_pipeline(scaler, sampler, pca, clf)


start_time = time.time()
pipe_f1.fit(X_train, y_train)
print(f"Time spent to fit: {time.time() - start_time:.6f} seconds")  

start_time = time.time()  
svm_f1_preds = pipe_f1.predict(X_test)
print(f"Time spent to test: {time.time() - start_time:.6f} seconds")

print(f"F1-score on test set: {100 * f1_score(y_test, svm_f1_preds):.3f}%")

## Παρουσίαση αποτελεσμάτων

Στους ακόλουθους πίνακες δείχνουμε την επίδοση των βελτιωμένων ταξινομητών στο test set. Η στήλη ΔAccuracy / ΔF1score δείχνει τη βελτίωση σε σχέση με τον OOTB ταξινομητή.

#### Αποτελέσματα accuracy

| **Metric**     | **Dummy** | **MLP** | **SVM** |
|:--------------:|:---------:|:-------:|:-------:|
| Accuracy (%)   | 83.994    | 78.435  | 87.711  |
| ΔΑccuracy (%)  |   -       | -7.631  | +2.968  |
| Train Time (ms)|   -       | 18339.9 | 730.376 |
| Test Time (ms) |   -       | 5.286   | 282.804 |

In [ ]:
mlp_names = ['MLP OOTB', 'MLP Opt.']
svm_names = ['SVM OOTB', 'SVM Opt.']
# Metrics of one execution (check the table above). MLP Metrics are stochastic.
mlp_acc = [85.976, 78.435] 
svm_acc = [84.473, 87.711] 
fig, ax = plt.subplots()
ax.set_ylabel("Accuracy (%)")
ax.bar(mlp_names, mlp_acc)
ax.bar(svm_names, svm_acc)
plt.show()

#### Αποτελέσματα F1-score

| **Metric**     | **Dummy** | **MLP** | **SVM** |
|:--------------:|:---------:|:-------:|:-------:|
| F1 Score (%)   |  0.000    | 50.664  | 53.183  |
| ΔF1 Score (%)  |   -       | +4.161  | +33.739 |
| Train Time (ms)|   -       | 3837.47 | 2054.35 |
| Test Time (ms) |   -       | 5.2323  | 348.814 |

In [ ]:
mlp_names = ['MLP OOTB', 'MLP Opt.']
svm_names = ['SVM OOTB', 'SVM Opt.']
# Metrics of one execution (check the table above). MLP Metrics are stochastic.
mlp_f1 = [46.503, 50.664] 
svm_f1 = [19.444, 53.183] 
fig, ax = plt.subplots()
ax.set_ylabel("F1-Score (%)")
ax.bar(mlp_names, mlp_f1)
ax.bar(svm_names, svm_f1)
plt.show()

#### Σχολιασμός MLP (Accuracy)

Παρατηρούμε πως η επίδοση που πετυχαίνουμε για το accuracy είναι χαμηλότερη από τον ootb ταξινομητή. Προσπαθήσαμε να δώσουμε περισσότερο βάρος στο preprocessing παρά το ίδιο το νευρωνικό. Στο νευρωνικό δίκτυο μειώσαμε το πλήθος νευρώνων (αν και αυξήσαμε το πλήθος των επιπέδων) σε σχέση με το ootb. Όπως βλέπουμε από τα αποτελέσματα, δεν πέτυχε η προσπάθεια αυτή. Στόχος ήταν να μην αυξήσουμε αλόγιστα τις παραμέτρους του νευρωνικού ώστε να εκπαιδεύεται γρήγορα.

Παρακάτω παραθέτουμε τους χρόνους εκπαίδευσης και ταξινόμησης του ootb MLP. Παρατηρούμε πως εκπαιδεύεται πιο γρήγορα από ότι η δική μας "optimized" εκδοχή, ωστόσο ο optimized ταξινομητής, ταξινομεί πιο γρήγορα το dataset αξιολόγησης.

In [ ]:
start_time = time.time()
clf = MLPClassifier()
clf.fit(X_train, y_train)
print(f"Time spent to fit: {time.time() - start_time:.6f} seconds") 

start_time = time.time()
_ = clf.predict(X_test)
print(f"Time spent to test: {time.time() - start_time:.6f} seconds") 

Διαισθητικά θα περιμέναμε η optimized εκδοχή να εκπαιδευτόταν γρηγορότερα λόγω του πλήθους παραμέτρων (στα κρυφά επίπεδα) των δύο νευρωνικών:

- (30 χαρακτηριστικά + 1 bias) * 100 νευρώνες = 3100 παράμετροι στο ootb

- (14 PCA συνιστώσες + 1 bias) * 20 νευρώνες στο 1ο κρυφό επίπεδο + (20 έξοδοι νευρώνων + 1 bias) * 12 νευρώνες στο 2ο κρυφό επίπεδο = 552 παράμετροι στην υλοποίησή μας

Ωστόσο, ίσως λόγω της πιο σύνθετης δομής του νευρωνικού μας (2 layers έναντι του ενός του ootb MLP) και του extensive preprocessing που εκτελούμε, να είναι αναμενόμενη η τόσο μεγάλη αύξηση στον χρόνο εκπαίδευσης.

Μπορεί να μην ξεπεράσαμε την επίδοση του ootb MLP, ωστόσο πετύχαμε αρκετά καλή επίδοση, δίνοντας βάρος όπως προείπαμε στο preprocessing. 
Αν μπορούσαμε να επαναλαμβάναμε τη διαδικασία εύρεσης παραμέτρων, θα θέταμε πιο χαλαρούς περιορισμούς στο πλήθος των κρυφών επιπέδων και ίσως και στο πλήθος των νευρώνων. Δεν θα ορίζαμε τη δομή του νευρωνικού αυστηρά με λίγες μόνο εκτελέσεις. (Πχ. με λίγες εκτελέσεις καταλήξαμε σε 2 κρυφά επίπεδα)

In [ ]:
fig, ax = plt.subplots(ncols = 2)
ConfusionMatrixDisplay.from_predictions(y_test, acc_preds[1], ax = ax[0], cmap = 'Purples', colorbar = False)
ax[0].set_title("MLP Confussion Matrix")
ConfusionMatrixDisplay.from_predictions(y_test, mlp_acc_preds, ax = ax[1], cmap = 'Purples', colorbar = False)
ax[1].set_title("Opt. Acc MLP Confussion Matrix")
plt.show()

Αν αξιολογήσουμε την επίδοση του ταξινομητή που φτιάξαμε και με το confusion matrix, πέρα απλώς από το accuracy, παρατηρούμε πως πλέον ο ταξινομητής μας προβλέπει πολύ πιο συχνά ταιριαστά ζευγάρια. Για να το πετύχει αυτό, καταλήγει να προβλέπει επίσης συχνά πως πολλά αταίριαστα ζευγάρια ταιριάζουν. Το γενικό συμπέρασμα πάντως, είναι πως το accuracy ενδεχομένως να είναι αρκετά "αυστηρή" μετρική στο πρόβλημα μας, διότι το dataset είναι αρκετά imbalanced. 

#### Σχολιασμός MLP (F1 - Score)

In [ ]:
fig, ax = plt.subplots()
ConfusionMatrixDisplay.from_predictions(y_test, mlp_f1_preds, ax = ax, cmap = 'Purples', colorbar = False)
ax.set_title("Opt. F1 MLP Confussion Matrix")
plt.show()

Το optimized κατά F1 MLP, τείνει να προβλέπει περισσότερα ζεύγη λάθος, από ότι ο F1 SVM τον οποίο και εξετάζουμε στη συνέχεια. Πετυχαίνει ικανοποιητικό αποτέλεσμα στα "ταιριαστά" ζευγάρια, ωστόσο προβλέπει πολύ πιο συχνά πως ταιριαστά ζευγάρια δεν ταιριάζουν και αντίστοιχα πως αταίριαστα ζευγάρια ταιριάζουν. Δεν το θεωρούμε επιθυμητό χαρακτηριστικό, διότι κρίνουμε εσφαλμένα αρκετά ζευγάρια, κάτι που σε ρεαλιστικές συνθήκες, μπορεί δυνητικά να απογοητεύσει τους συμμετέχοντες. Η αύξηση στο F1 Score είναι φανερή όχι όμως τόσο σημαντική όπως στην αντίστοιχη του SVM, όμως χρησιμοποιούμε αρκετά μικρότερο δίκτυο (σε αντίθεση με το ootb) για να πετύχουμε καλύτερο αποτέλεσμα από την ootb επίδοση. Έτσι έχουμε λογικό χρόνο εκπαίδευσης. Βλέπουμε επίσης πως έχουμε πολύ γρήγορο χρόνο ταξινόμησης.

Το ποσοστό των κλάσεων:

In [ ]:
print(f"Πλήθος Κλάσεων: 2")
freqs = (100 * y_test.value_counts() / y_test.size).map("{:.2f}%".format)
print(f"Συχνότητα Δειγμάτων Κλάσεων:\n{freqs}")

#### Σχολιασμός SVM

In [ ]:
fig, ax = plt.subplots(ncols = 2)
ConfusionMatrixDisplay.from_predictions(y_test, acc_preds[-1], ax = ax[0], cmap = 'Purples', colorbar = False)
ax[0].set_title("SVM Confussion Matrix")
ConfusionMatrixDisplay.from_predictions(y_test, svm_acc_preds, ax = ax[1], cmap = 'Purples', colorbar = False)
ax[1].set_title("Opt. Acc SVM Confussion Matrix")
plt.show()

Το accuracy εδώ βελτιώνεται μόνο κατά 3%. Αυτή η μικρή αλλαγή επιβεβαιώνεται και από το confusion matrix. Κάνουμε λιγότερα false negative σφάλματα και πετυχαίνουμε αισθητά περισσότερα true positive, με το μικρό κόστος να χάνουμε ελάχιστα true negative. Παρακάτω, ο SVM με βελτίωση στο F1 score  πετυχαίνει περισσότερα true positive, αλλά αυτός εδώ ο ταξινομητής είναι καλός αν θέλουμε να διατηρήσουμε ακρίβεια στα true negative.

In [ ]:
fig, ax = plt.subplots(ncols = 2)
ConfusionMatrixDisplay.from_predictions(y_test, acc_preds[-1], ax = ax[0], cmap = 'Purples', colorbar = False)
ax[0].set_title("SVM Confussion Matrix")
ConfusionMatrixDisplay.from_predictions(y_test, svm_f1_preds, ax = ax[1], cmap = 'Purples', colorbar = False)
ax[1].set_title("Opt. F1 SVM Confussion Matrix")
plt.show()

Παραπάνω είχαμε ορίσει ως `pos_label` το match = 1, δηλαδή δύο άνθρωποι να ταιριάζουν. Πολλές φορές στο F1, είθισται να βάζουμε ως pos_label τη σπάνια περίπτωση που μας ενδιαφέρει να εντοπίσουμε. Όπως βλέπουμε παραπάνω, από τα δύο confusion matrices που παραθέτουμε, ο optimized F1 ταξινομητής προβλέπει πιο συχνά ζευγάρια τα οποία στη πράξη ταιριάζουν. Ωστόσο, για να το πετύχει αυτό, τείνει να προβλέπει πιο συχνά πως ένα ζεύγαρι ανθρώπων ταιριάζει. Το trade off αυτό είναι επιθυμητό διότι κύριος στόχος μας είναι να προβλέπουμε όσα περισσότερα "ταιριαστά" ζευγάρια.

## Συμπεράσματα

Στις επιλογές που κάνουμε παρακάτω, αγνοούμε τον dummy classifier. Σαφώς θα είναι ο πιο ταχύς ταξινομητής τόσο στην εκπαίδευση και στην αξιολόγηση, ωστόσο μας ενδιαφέρουν οι άλλοι ταξινομητές που αποδίδουν ικανοποιητικά και σε accuracy και σε F1 score.

- Ως προς Accuracy: Optimized (ως προς Accuracy) SVM
- Ως προς F1: Optimized (ως προς F1) SVM
- Ως προς χρόνο εκπαίδευσης : Optimized (ως προς Accuracy) SVM
- Ως προς χρόνο ταξινόμησης : Optimized (ως προς F1) MLP 
- Ως προς γενική συμπεριφορά από το confusion matrix (τείνει να ταιριάζει περισσότερα ζευγάρια (πιο "loose") ή όχι (πιο "conservative")): <br>
Επειδή το dataset εκπαίδευσης είναι imbalanced, ταξινομητές που τείνουν να αυξήσουν το accuracy, τείνουν να είναι και πιο conservative. <br>
Από την άλλη, ταξινομητές που τείνουν να αυξήσουν το F1 (με `pos_label` το `match` = 1) τείνουν να ταιριάζουν περισσότερα ζευγάρια. <br> 
Ωστόσο, αυτή η συσχέτιση "συμπεριφοράς" και μετρικών, εξαρτάται σε μεγάλο βαθμό από το imbalance που προαναφέραμε.

Ως γενικό performance επιλέγουμε το **SVM** (σε οποιαδήποτε από τις δύο optimized εκδοχές του). Εάν μας ενδιαφέρει γρήγορη απόκριση και ικανοποιητικό F1 score, (αν χρησιμοποιηθεί ο ταξινομητής μας σε κάποια real-time εφαρμογή, όπως πχ. το `Tinder` για "ταίριασμα" ανθρώπων, θα μας ενδιέφερε να αποκρινόταν γρήγορα) τότε θα προτείναμε τον optimized ως προς **F1 MLP**.

Σαν πρώτο σχόλιο για τον SVM, είναι λογικό που προκύπτουν μικρές τιμές `C` στις βέλτιστες εκδόσεις, διότι τα περισσότερα δεδομένα εισόδου είναι η άποψη των συμμετεχόντων κι όχι κάποια αντικειμενική φυσική ποσότητα. Με άλλα λόγια, περιμένουμε να έχουμε πολλούς outliers και το μικρό `C` το αντιμετωπίζει αυτό. Επίσης, για τον ίδιο λόγο το task της ταξινόμησης είναι απαιτητικό, άρα το MLP δύσκολα μπορεί να βρει μία επιφάνεια διαχωρισμού. Στο SVM δοκιμάζονται πολλές τεχνικές διαχωρισμού, με επιτυχία. Ίσως χρήση άλλων activation functions πέρα από τη ReLU, να προσέδιδε καλύτερη μορφή στις επιφάνειες απόφασης του MLP.